In [ ]:
%matplotlib qt
import mne
from mne.preprocessing import create_ecg_epochs, create_eog_epochs
import numpy as np
import pandas as pd
import seaborn as sns
import os
import pickle
import os.path as op
import matplotlib
from tqdm.notebook import tqdm
import time
from itertools import product
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
from mne.coreg import Coregistration
from mne.datasets import fetch_fsaverage
from mne.minimum_norm import make_inverse_operator, apply_inverse, apply_inverse_epochs
from mne_connectivity import spectral_connectivity_epochs
from scipy import stats as stats
from mne.stats import spatio_temporal_cluster_1samp_test, summarize_clusters_stc
import statsmodels.api as sm
from scipy.stats import shapiro, levene, ttest_ind, f_oneway
from statsmodels.formula.api import ols
from statsmodels.stats.anova import AnovaRM
from statsmodels.stats.multicomp import MultiComparison
import statsmodels.formula.api as smf
from sklearn.cross_decomposition import CCA
from nilearn.plotting import plot_connectome

Loading MEG data (movement corrected) and defining events

In [ ]:
# loading subject IDs (subject 697 cant be loaded and subject 859 has different dev_head_t during the two recordings)
subjects_fname = '/Users/payamsadeghishabestari/KI_MEG/sub_date.txt'
subject_ids = np.loadtxt(fname=subjects_fname, delimiter=',', skiprows=1, usecols=1)
subject_ids = [int(s_id) for s_id in subject_ids]

# find all the subject folders
directory = '/Users/payamsadeghishabestari/KI_MEG/meg_rec_tinmeg1' 
folders_list = []
for folder in sorted(os.listdir(directory)): ## iterate over folders in that directory
    f = os.path.join(directory, folder)
    if os.path.isdir(f): ## select only folders
        folders_list.append(f)

# create a dictionary of subjects with their files
files_dict = {}
for subject_id in subject_ids:
    for folder in folders_list:
        if f'{subject_id}' in folder:
            f = os.path.join(folder, sorted(os.listdir(folder))[-1])
            files_dict[f'{subject_id}'] = [f]

In [ ]:
# creating event dictionary
# tinmeg1
keys = ['PO60_70', 'PO60_75', 'PO60_80', 'PO60_85', 'PO60_90', 'PO60_95',
        'PO70_75', 'PO70_80', 'PO70_85', 'PO70_90', 'PO70_95',
        'GP60_i0', 'GP60_i60', 'GP60_i120', 'GP60_i240',
        'GP70_i0', 'GP70_i60', 'GP70_i120', 'GP70_i240',
        'GO_60', 'GO_70']
values = [40968, 36872, 34824, 33800, 33288, 33032,
        36876, 34828, 33804, 33292, 33036,
        49800, 49736, 49704, 49688,
        49804, 49740, 49708, 49692,
        16386, 16390]
events_dict_tinmeg1 = {}
for key, value in zip(keys, values):
        events_dict_tinmeg1[key] = value

# tinmeg2
keys = ['GPP_00', 'GPG_00', 'PO_00', 'GO_00', 'PPP_00', 'PPG_00',
        'GPP_03', 'GPG_03', 'PO_03', 'GO_03',
        'GPP_08', 'GPG_08', 'PO_08', 'GO_08',
        'GPP_30', 'GPG_30', 'PO_30', 'GO_30',
        'GPP_33', 'GPG_33', 'PO_33', 'GO_33',
        'GPP_38', 'GPG_38', 'PO_38', 'GO_38',
        'GPP_80', 'GPG_80', 'PO_80', 'GO_80',
        'GPP_83', 'GPG_83', 'PO_83', 'GO_83',
        'GPP_88', 'GPG_88', 'PO_88', 'GO_88']
values = [1, 2, 4, 8, 16, 32,
        49, 50, 52, 56,
        33, 34, 36, 40,
        193, 194, 196, 200,
        241, 242, 244, 248,
        225, 226, 228, 232,
        129, 130, 132, 136,
        177, 178, 180, 184,
        161, 162, 164, 168]
events_dict_tinmeg2 = {}
for key, value in zip(keys, values):
        events_dict_tinmeg2[key] = value

#tinmeg3
keys = ['GPP_00', 'GPG_00', 'PO_00', 'GO_00', 'PPP_00', 'PPG_00',
        'GPP_03', 'GPG_03', 'PO_03', 'GO_03',
        'GPP_08', 'GPG_08', 'PO_08', 'GO_08']
values = [1, 2, 4, 8, 16, 32,
        49, 50, 52, 56,
        33, 34, 36, 40]
events_dict_tinmeg3 = {}
for key, value in zip(keys, values):
        events_dict_tinmeg3[key] = value

Maxwell Filtering and environmental noise reduction (if necessary)

In [ ]:
# loading the empty room recordings before and after exp
fname_empty_before = '/Users/payamsadeghishabestari/KI_MEG/697/empty_room_before.fif'
fname_empty_after = '/Users/payamsadeghishabestari/KI_MEG/697/empty_room_after.fif'
raw_empty_before = mne.io.read_raw_fif(fname=fname_empty_before, preload=True, allow_maxshield=True, verbose=False)
raw_empty_after = mne.io.read_raw_fif(fname=fname_empty_after, preload=True, allow_maxshield=True, verbose=False)

# compute projections for empty room recordings and concatenate them
raw_empty_before.del_proj()
raw_empty_after.del_proj()
empty_room_before_projs = mne.compute_proj_raw(raw_empty_before, n_grad=2, n_mag=2, verbose=False)
empty_room_after_projs = mne.compute_proj_raw(raw_empty_after, n_grad=2, n_mag=2, verbose=False)
extended_proj = list(np.concatenate((np.array(empty_room_before_projs), np.array(empty_room_after_projs))))

# load the experiment recording
fname = '/Users/payamsadeghishabestari/KI_MEG/697/tinmeg1-1.fif'
raw = mne.io.read_raw_fif(fname=fname, preload=True, allow_maxshield=True, verbose=False)

# estimating continous head movement
chpi_freqs, ch_idx, chpi_codes = mne.chpi.get_chpi_info(info=raw.info)
chpi_amplitudes = mne.chpi.compute_chpi_amplitudes(raw)
chpi_locs = mne.chpi.compute_chpi_locs(raw.info, chpi_amplitudes)
head_pos = mne.chpi.compute_head_pos(raw.info, chpi_locs, verbose=True)

# find bad channels
noisy_chs, flat_chs = mne.preprocessing.find_bad_channels_maxwell(raw, head_pos=head_pos, verbose=True)
bads = raw.info["bads"] + noisy_chs + flat_chs
raw.info["bads"] = bads

# apply movement corection and time-signal space seperation
raw_sss = mne.preprocessing.maxwell_filter(raw, head_pos=head_pos, st_fixed=True,
                                            extended_proj=extended_proj,verbose=True)

In [ ]:
fname = '/Users/payamsadeghishabestari/KI_MEG/697/tinmeg1.fif'
raw = mne.io.read_raw_fif(fname=fname, preload=True, allow_maxshield=True, verbose=False)

# estimating continous head movement
chpi_freqs, ch_idx, chpi_codes = mne.chpi.get_chpi_info(info=raw.info)
chpi_amplitudes = mne.chpi.compute_chpi_amplitudes(raw)
chpi_locs = mne.chpi.compute_chpi_locs(raw.info, chpi_amplitudes)
head_pos = mne.chpi.compute_head_pos(raw.info, chpi_locs, verbose=True)

# find bad channels
noisy_chs, flat_chs = mne.preprocessing.find_bad_channels_maxwell(raw, head_pos=head_pos, verbose=True)
bads = raw.info["bads"] + noisy_chs + flat_chs
raw.info["bads"] = bads

# apply movement corection and time-signal space seperation
raw_sss = mne.preprocessing.maxwell_filter(raw, head_pos=head_pos, st_fixed=True, verbose=True)

Preprocessing

In [ ]:
sfreq = 250
(l_freq, h_freq) = (0.1, 40)
(tmin, tmax) = (-0.3, 0.3) # baseline period of 300 ms
reject_criteria = dict(grad=4000e-13, mag=4e-12, eog=250e-6)    # T/m  # T  # V
flat_criteria = dict(mag=1e-15, grad=1e-13)  # 1 fT  # 1 fT/cm  (adding flat option to eog)
subjects = list(files_dict.keys())[:]

# reading the MEG file
for subject in tqdm(subjects): 
    start_time = time.time()
    print(subject)
    print('reading the MEG file, be patient ...')
    fname = files_dict[subject][0]
    raw = mne.io.read_raw_fif(fname=fname, preload=True, allow_maxshield=True, verbose=False)
    #if raw.last_samp / raw.info['sfreq'] < 3000:
    #    raise ValueError(f'All .fif files might not be loaded for subject {subject}')
    events_orig = mne.find_events(raw, stim_channel=None, min_duration=0.005, shortest_event=1, uint_cast=True, verbose=False) # min_duration = 0

    # delay compensation for tinmeg1 data
    #delay = int((50 / 1000) * raw.info['sfreq']) # 50 ms delay 
    #po_ids = list(events_dict_tinmeg3.values())[:11] # only PO triggers
    #for row in range(len(events_orig)):
    #    if events_orig[row][2] in po_ids:
    #        events_orig[row][0] = events_orig[row][0] - delay

    # resampling and filtering the data
    print('resampling and filtering the data, be patient, will last a while ...')
    raw, events = raw.resample(sfreq=sfreq, events=events_orig, verbose=False)
    raw = raw.filter(l_freq=l_freq, h_freq=h_freq, verbose=False) 

    # creating ECG and EOG evoked responses
    ecg_evoked_meg,  ecg_evoked_grad = create_ecg_epochs(raw,
                                    verbose=False).average().apply_baseline(baseline=(None, -0.2),
                                    verbose=False).plot_joint(picks=['meg', 'grad'], show=False)
    eog_evoked_meg,  eog_evoked_grad = create_eog_epochs(raw,
                                    verbose=False).average().apply_baseline(baseline=(None, -0.2),
                                    verbose=False).plot_joint(picks=['meg', 'grad'], show=False)

    # computing ICA and remove ECG, saccade and muscle artifacts (if any) and interpolating (if any)
    print('computing ICA (this might take a while) ...')
    ica = mne.preprocessing.ICA(n_components=0.95, max_iter=800, method='infomax',
                                random_state=42, fit_params=dict(extended=True)) 
    ica.fit(raw, verbose=False) 
    ecg_indices, ecg_scores = ica.find_bads_ecg(raw, method="ctps", measure='zscore', verbose=False)
    if len(ecg_indices) > 0:
        ecg_component = ica.plot_properties(raw, picks=ecg_indices, verbose=False, show=False)
    emg_indices, emg_scores = ica.find_bads_muscle(raw, verbose=False)
    if len(emg_indices) > 0:
        emg_component = ica.plot_properties(raw, picks=emg_indices, verbose=False, show=False)
    eog_indices, eog_scores = ica.find_bads_eog(raw, ch_name='EOG002') 
    if len(eog_indices) > 0:
        eog_component = ica.plot_properties(raw, picks=eog_indices, verbose=False, show=False)

    exclude_idxs = ecg_indices + emg_indices
    ica.apply(raw, exclude=exclude_idxs, verbose=False)
    raw.interpolate_bads(verbose=False)

    # event dict selection, epoching and dropping bad epochs 
    #if fname[-43] == '1':
    #    events_dict = events_dict_tinmeg1
    #if fname[-43] == '2':
    #    events_dict = events_dict_tinmeg2
    #if fname[-43] == '3':
    #    events_dict = events_dict_tinmeg3
    
    events_dict = events_dict_tinmeg1
    print('Epoching data ...')
    epochs = mne.Epochs(raw, events, event_id=events_dict, tmin=tmin, tmax=tmax, baseline=(None, 0),
                        reject=None, flat=None, preload=True, verbose=False) 
    dropped_epochs_fig = epochs.plot_drop_log(color=(0.6, 0.2, 0.4), width=0.4, show=False)

    # creating a report
    report = mne.Report(title=f'report_subject_{subject}', verbose=False)
    report.add_raw(raw=raw, title='recording after preprocessing', butterfly=False, psd=False) 
    report.add_figure(fig=ecg_evoked_meg, title='ECG evoked MEG', image_format='PNG')
    report.add_figure(fig=ecg_evoked_grad, title='ECG evoked Gradiometer', image_format='PNG')
    report.add_figure(fig=eog_evoked_meg, title='EOG evoked MEG', image_format='PNG')
    report.add_figure(fig=eog_evoked_grad, title='EOG evoked Gradiometer', image_format='PNG')
    if len(ecg_indices) > 0:
        report.add_figure(fig=ecg_component, title='ECG component', image_format='PNG')
    if len(emg_indices) > 0:
        report.add_figure(fig=emg_component, title='EMG component', image_format='PNG')
    if len(eog_indices) > 0:
        report.add_figure(fig=eog_component, title='EOG component (saccade)', image_format='PNG')    
    report.add_figure(fig=dropped_epochs_fig, title='Dropped Epochs', image_format='PNG')
    
    # saving report and epochs
    
    fname_report = f'/Users/payamsadeghishabestari/KI_MEG/pending files/{subject}/report_subject_{subject}.html'
    fname_epoch = f'/Users/payamsadeghishabestari/KI_MEG/pending files/{subject}/epochs_subject_{subject}-epo.fif'
    report.save(fname=fname_report, open_browser=False, overwrite=True, verbose=False)
    epochs.save(fname=fname_epoch, overwrite=True, verbose=False)
    print(f'elapsed time for subject {subject} was {time.time() - start_time}')

Concatenating, creating evoked objects and grand averaging

In [ ]:
# Create epochs dictionary (some needs concatenating)
epochs_folder = '/Users/payamsadeghishabestari/KI_MEG/epochs/tinmeg1'
epochs_file = {}
for f in sorted(os.listdir(epochs_folder)):
    file = os.path.join(epochs_folder, f)
    if file.endswith("-epo.fif") and '697' not in file and '750' not in file and '853' not in file and '841' not in file:
        epochs_file[f'{file[-11:-8]}'] = file

# compute evoked objects, and making grand average dictionary
evs = []
for ep_f in tqdm(list(epochs_file.values())):
    evs.append(mne.read_epochs(fname=ep_f, verbose=False).average(picks=['meg', 'eog'], by_event_type=True))

grnd_ev_dict = {}
for stim_idx, stim in enumerate(list(events_dict_tinmeg1.keys())):
    evs_stim = []
    for ev in evs:
        evs_stim.append(ev[stim_idx])
    grnd_ev_dict[stim] = evs_stim

grand_ev_dict = {}
for stim in list(grnd_ev_dict.keys()):
    grand_ev_dict[stim] = mne.grand_average(grnd_ev_dict[stim])

Check EOG response (Niklas thesis)

In [ ]:
#### PO at 60 dB
fig, axs = plt.subplots(1, 1, figsize=(8, 5))
time_array = np.linspace(-300, 300, 151)
stims = ['PO60_70', 'PO60_75', 'PO60_80', 'PO60_85', 'PO60_90', 'PO60_95']
colors = ['purple', 'brown', 'blue', 'orange', 'red', 'green']
for stim, color in zip(stims, colors):
    axs.plot(time_array, grand_ev_dict[stim].get_data(picks='EOG002')[0] * 1e6, label=stim)
axs.axvspan(50, 220, alpha=0.4, color='lightcyan')
axs.legend(fontsize=9, frameon=False)
axs.spines['top'].set_visible(False); axs.spines['right'].set_visible(False)
axs.grid(axis='x', color='k', linestyle='--', linewidth=0.5)
axs.vlines(0, -50, 10, colors='black',linestyles='--')
axs.set_ylabel(f'EOG amplitude at 60 dB (µv)')
axs.set_xlabel(f'Time (ms)')

#### GP at 60 dB
fig, axs = plt.subplots(1, 1, figsize=(8, 5))
time_array = np.linspace(-300, 300, 151)
stims = ['GP60_i0', 'GP60_i60', 'GP60_i120', 'GP60_i240']
colors = ['blue', 'orange', 'red', 'green']
for stim, color in zip(stims, colors):
    axs.plot(time_array, grand_ev_dict[stim].get_data(picks='EOG002')[0] * 1e6, label=stim)
axs.axvspan(50, 220, alpha=0.4, color='lightcyan')
axs.legend(fontsize=9, frameon=False)
axs.spines['top'].set_visible(False); axs.spines['right'].set_visible(False)
axs.grid(axis='x', color='k', linestyle='--', linewidth=0.5)
axs.vlines(0, -50, 10, colors='black',linestyles='--')
axs.set_ylabel(f'EOG amplitude at 60 dB (µv)')
axs.set_xlabel(f'Time (ms)')
axs.set_xlim([-300, 300])


Check ERFs (Niklas thesis)

In [ ]:
# separate chanels on left and right
info_ch = evs[0][0].info['chs']
meg_chs_right = []; meg_chs_left = []
grad_chs_right = []; grad_chs_left = []
for i in range(len(info_ch)):
    if info_ch[i]['unit'] == 112: # meg code
        if info_ch[i]['loc'][0] > 0:
            meg_chs_right.append(info_ch[i]['ch_name'])
        if info_ch[i]['loc'][0] < 0:
            meg_chs_left.append(info_ch[i]['ch_name'])
    if info_ch[i]['unit'] == 201: # grad code
        if info_ch[i]['loc'][0] > 0:
            grad_chs_right.append(info_ch[i]['ch_name'])
        if info_ch[i]['loc'][0] < 0:
            grad_chs_left.append(info_ch[i]['ch_name'])

# select the left/ channels with largest ptp amplitude
ev_data_left = grand_ev_dict['PO60_70'].get_data(picks=grad_chs_left)
ev_data_right = grand_ev_dict['PO60_70'].get_data(picks=grad_chs_right)
max_values = []
for ch_idx in range(len(ev_data_left)):
    max_values.append(ev_data_left[ch_idx][50:150].max())
ch_max_left = grad_chs_left[np.argmax(np.array(max_values))]
max_values = []
for ch_idx in range(len(ev_data_right)):
    max_values.append(ev_data_right[ch_idx][50:150].max())
ch_max_right = grad_chs_right[np.argmax(np.array(max_values))]

In [ ]:
# PO 60 dB
fig, axs = plt.subplots(1, 1, figsize=(10, 3))
time_array = np.linspace(-300, 300, 151)
stims = ['PO60_70', 'PO60_75', 'PO60_80', 'PO60_85', 'PO60_90', 'PO60_95']
alphas = np.linspace(0.2, 1, 6)
for stim, alpha in zip(stims, alphas):
    axs.plot(time_array, grand_ev_dict[stim].get_data(picks=ch_max_left)[0] * 1e13, color='#1f77b4', alpha=alpha, label=stim)
    axs.plot(time_array, grand_ev_dict[stim].get_data(picks=ch_max_right)[0] * 1e13, color='red', alpha=alpha, label=stim)
axs.axvspan(50, 150, alpha=0.4, color='lightcyan')
axs.spines['top'].set_visible(False); axs.spines['right'].set_visible(False)
axs.grid(axis='x', color='k', linestyle='--', linewidth=0.5)
axs.set_ylabel(f'ERF amplitude at 60 dB (fT/m)')
axs.set_xlabel(f'Time (ms)')
axs.set_xlim([-300, 300])

# GP 60 dB
fig, axs = plt.subplots(1, 1, figsize=(10, 3))
time_array = np.linspace(-300, 300, 151)
stims = ['GP60_i0', 'GP60_i60', 'GP60_i120', 'GP60_i240']
dc_shifts = [0, 50, 100, 150]
for stim, dc_shift in zip(stims, dc_shifts):
    axs.plot(time_array, dc_shift + grand_ev_dict[stim].get_data(picks=ch_max_left)[0] * 1e13, color='#1f77b4', label=stim)
    axs.plot(time_array, dc_shift + grand_ev_dict[stim].get_data(picks=ch_max_right)[0] * 1e13, color='red', label=stim)
axs.axvspan(50, 150, alpha=0.4, color='lightcyan')
axs.spines['top'].set_visible(False); axs.spines['right'].set_visible(False)
axs.grid(axis='x', color='k', linestyle='--', linewidth=0.5)
axs.set_ylabel(f'Relative values')
axs.set_xlabel(f'Time (ms)')
axs.set_xlim([-300, 300])

Source Localization

In [ ]:
# Some Notes:
# 1. check with and without ICA, since we didnt remove blink, however makes sense to proceed without applying ICA
# 2. before the time 0, we have an activity at GP series so its not a good idea to use epochs baseline as a time span to estimate noise covariance
# the good way to estimate the noise cov is to use only PO series baseline   

In [ ]:
# Cortical surface reconstruction (+bem, +head_model) with FreeSurfer (as an example for one subject)
$ export FREESURFER_HOME=/Applications/freesurfer/7.4.1
$ export SUBJECTS_DIR=$FREESURFER_HOME/subjects
$ source $FREESURFER_HOME/SetUpFreeSurfer.sh
$ recon-all -s 0863 -i /Users/payamsadeghishabestari/KI_MEG/MRI/0863/00000003/00000001.dcm 
$ recon-all -all -subjid 0863

# setting up watershed BEM files 
subject = '0863'
subjects_dir = '/Applications/freesurfer/7.4.1/subjects'
mne.bem.make_watershed_bem(subject, subjects_dir=None,
                            overwrite=False, volume='T1', atlas=False,
                            gcaatlas=False, preflood=None, show=False,
                            copy=True, T1=None, brainmask='ws.mgz', verbose=None)

# mri_convert /Users/payamsadeghishabestari/nilearn_data/schaefer_2018/Schaefer2018_400Parcels_7Networks_order_FSLMNI152_1mm.nii.gz $SUBJECTS_DIR/0750/mri/schaefer_atlas.mgz
# bbregister --s 0750 --mov $SUBJECTS_DIR/0750/mri/schaefer_atlas.mgz --t1 --reg $SUBJECTS_DIR/0750/schaefer.reg
# mri_aparc2aseg --s 0750 --annot $SUBJECTS_DIR/0750/mri/schaefer_atlas.mgz
# mri_label2vol --seg $SUBJECTS_DIR/0750/mri/aparc+aseg.mgz --temp $SUBJECTS_DIR/0750/mri/orig.mgz --o schaefer_combined.mgz --reg schaefer.reg --fillthresh 0.5



In [ ]:
# po_stims = ['PO60_70', 'PO60_75', 'PO60_80', 'PO60_85', 'PO60_90', 'PO60_95'] # for tinmeg1
po_stims = ['PO_00', 'PPP_00', 'PO_03', 'PO_08'] # tinmeg3
# po_stims = ['PO_00', 'PO_03', 'PO_08', 'PO_30', 'PO_33', 'PO_38', 'PO_80', 'PO_83', 'PO_88'] # tinmeg2

subjects_dir = '/Applications/freesurfer/7.4.1/subjects'
method = "dSPM"
snr = 3.0
lambda2 = 1.0 / snr**2

for subject in np.array(list(epochs_file.keys()))[[11,12]]: # list(epochs_file.keys())
    
    subject_idx = list(epochs_file.keys()).index(subject)
    report = mne.Report(title=f'source_localization_report_subject_{subject}', verbose=False)
    
    # Setting up the surface source space
    print(f'Setting up bilateral hemisphere surface-based source space with subsampling for subject {subject} ...')
    src = mne.setup_source_space(f'{subject}', spacing="oct6", subjects_dir=subjects_dir, n_jobs=-1, verbose=None)

    # Setting up the boundary-element model (BEM) 
    print(f'Creating a BEM model for subject ...')
    bem_model = mne.make_bem_model(subject=f'{subject}', ico=4, subjects_dir=subjects_dir, verbose=False)  
    bem = mne.make_bem_solution(bem_model, verbose=False)
    report.add_bem(subject=f'{subject}', subjects_dir=subjects_dir, title="MRI & BEM", decim=10, width=512)

    # Aligning coordinate frame (coregistration MEG-MRI)
    print(f'Coregistering MRI with a subjects head shape ...')
    # info = grnd_ev_dict['PO60_80'][subject_idx].info # tinmeg1
    info = grnd_ev_dict['PO_00'][subject_idx].info # tinmeg3
    coreg = Coregistration(info, f'{subject}', subjects_dir, fiducials='auto')
    coreg.fit_fiducials(verbose=False)
    coreg.fit_icp(n_iterations=40, nasion_weight=2.0, verbose=False) # refining with ICP
    coreg.omit_head_shape_points(distance=5.0 / 1000) # omitting bad points (larger than 5mm)
    coreg.fit_icp(n_iterations=40, nasion_weight=10, verbose=False) # final fitting
    fname_trans = f'/Users/payamsadeghishabestari/KI_MEG/trans/{subject}-trans.fif'
    mne.write_trans(fname_trans, coreg.trans, overwrite=True, verbose=False)
    report.add_trans(trans=fname_trans, info=info, subject=f'{subject}',
                    subjects_dir=subjects_dir, alpha=1.0, title="Co-registration")

    # Computing the forward solution
    print(f'Computing the forward solution ...')
    fwd = mne.make_forward_solution(info, trans=coreg.trans, src=src, bem=bem, meg=True,
                                    eeg=False, mindist=5.0, n_jobs=None, verbose=False)

    # Computing the regularized noise-covariance matrix (consider the notes)
    print(f'Estimate the noise covariance of the recording ...')
    epochs = mne.read_epochs(fname=epochs_file[subject], verbose=False)
    noise_cov = mne.compute_covariance(epochs[po_stims], tmax=0.0, method=("empirical", "shrunk"),
                                        verbose=False) # using the epochs baseline 
    
    # Computing the minimum-norm inverse solution
    print(f'Computing the minimum-norm inverse solution ...')
    inverse_operator = make_inverse_operator(info, fwd, noise_cov, loose=0.2, depth=0.8, verbose=False)

    # Compute source estimate object
    print(f'Computing and saving the source estimate object ...')
    for key_id in list(grnd_ev_dict.keys()):
        stc = apply_inverse(grnd_ev_dict[key_id][subject_idx], inverse_operator, lambda2, method=method, pick_ori=None,
                            return_residual=False, verbose=False)
        fname_stc = f'/Users/payamsadeghishabestari/KI_MEG/stcs/tinmeg3/{subject}_{key_id}'
        stc.save(fname=fname_stc, overwrite=True, verbose=False)

    # saving report
    fname_report = f'/Users/payamsadeghishabestari/KI_MEG/reports/source_localization_report_subject_{subject}.html'
    report.save(fname=fname_report, open_browser=False, overwrite=True, verbose=False)

Morphing to freesurfer template brain

In [ ]:
subjects_dir = '/Applications/freesurfer/7.4.1/subjects'
fname_fsaverage_src = '/Users/payamsadeghishabestari/mne_data/MNE-fsaverage-data/fsaverage/bem/fsaverage-ico-5-src.fif'
directory = '/Users/payamsadeghishabestari/KI_MEG/stcs/tinmeg3' 
src_to = mne.read_source_spaces(fname_fsaverage_src)

# iterate over files in that directory
stc_files_list = []
for filename in sorted(os.listdir(directory)): 
    f = os.path.join(directory, filename)
    # checking if it is a file
    if os.path.isfile(f) and f.endswith("-lh.stc"): # or -rh
        stc_files_list.append(f)

# morphing from oct-6 to ico-5 at fsaverage
for stc_file in tqdm(stc_files_list):
    stc = mne.read_source_estimate(fname=stc_file, subject=f'{stc_file[50:54]}')
    
    morph = mne.compute_source_morph(stc, subject_from=f'{stc_file[50:54]}',
                                    subject_to="fsaverage", subjects_dir=subjects_dir,
                                    src_to=src_to)
    stc_morph = morph.apply(stc)
    fname_stc_morph = ''.join([stc_file[:49], '_morphed', stc_file[49:]])
    stc_morph.save(fname=fname_stc_morph, overwrite=True, verbose=False)


Visualization example

In [ ]:
# single subject
subject = '859'
event = 'PO60_90'
fname_stc = f'/Users/payamsadeghishabestari/KI_MEG/stcs/tinmeg1/{subject}_{event}-rh.stc'
stc = mne.read_source_estimate(fname=fname_stc, subject=f'0{subject}')
subjects_dir = '/Applications/freesurfer/7.4.1/subjects'
colormap = "viridis"
clim = dict(kind="value", lims=[4, 8, 12])
fig_brain = stc.plot(views="lat", hemi="split", size=(800, 400), subject=f'0{subject}',
                    subjects_dir=subjects_dir, background="w", colorbar=True, clim=clim,
                    colormap=colormap, time_viewer=True, show_traces=True)
fig_brain.add_annotation("aparc", borders=True, alpha=0.6)

In [ ]:
# average of subjects
event = 'GO_60'
directory = '/Users/payamsadeghishabestari/KI_MEG/stcs/tinmeg1_morphed'
stc_files_list = []
for filename in sorted(os.listdir(directory)): 
    f = os.path.join(directory, filename)
    # checking if it is a file
    if os.path.isfile(f) and f.endswith("-lh.stc") and (event in f): # or -rh
        stc_files_list.append(f)
data = []
for stc_file in tqdm(stc_files_list):
    stc = mne.read_source_estimate(fname=stc_file, subject='fsaverage')
    data.append(stc.data)
stc_average = mne.SourceEstimate(data=np.mean(np.array(data), axis=0), vertices=stc.vertices,
                                tmin=stc.tmin, tstep=stc.tstep, subject='fsaverage')
colormap = "viridis"
clim = dict(kind="value", lims=[4, 8, 12])
fig_brain = stc_average.plot(views="lat", hemi="split", size=(800, 400), subject='fsaverage',
                    subjects_dir=None, background="w", colorbar=True, clim=clim,
                    colormap=colormap, time_viewer=True, show_traces=True)

In [ ]:
# average of subjects
event1 = 'PO60_90'
event2 = 'GO_60'
event3 = 'GP60_i0'
event4 = 'GP60_i60'
event5 = 'GP60_i120'
event6 = 'GP60_i240'
subject = '853'

directory = '/Users/payamsadeghishabestari/KI_MEG/stcs/tinmeg1_morphed'
stc_files_list_1 = []; stc_files_list_2 = []; stc_files_list_3 = []; stc_files_list_4 = []; stc_files_list_5 = []; stc_files_list_6 = []
for filename in sorted(os.listdir(directory)): 
    f = os.path.join(directory, filename)
    # checking if it is a file
    if os.path.isfile(f) and f.endswith("-lh.stc") and (event1 in f) and (subject in f): # or -rh
        stc_files_list_1.append(f)
    if os.path.isfile(f) and f.endswith("-lh.stc") and (event2 in f) and (subject in f): # or -rh
        stc_files_list_2.append(f)
    if os.path.isfile(f) and f.endswith("-lh.stc") and (event3 in f) and (subject in f): # or -rh
        stc_files_list_3.append(f)
    if os.path.isfile(f) and f.endswith("-lh.stc") and (event4 in f) and (subject in f): # or -rh
        stc_files_list_4.append(f)
    if os.path.isfile(f) and f.endswith("-lh.stc") and (event5 in f) and (subject in f): # or -rh
        stc_files_list_5.append(f)
    if os.path.isfile(f) and f.endswith("-lh.stc") and (event6 in f) and (subject in f): # or -rh
        stc_files_list_6.append(f)

fig, axs = plt.subplots(1, 6, figsize=(17, 2))
ymax = 4
data_lh = []
data_rh = []
for stc_file in tqdm(stc_files_list_1):
    stc = mne.read_source_estimate(fname=stc_file, subject='fsaverage')
    data_lh.append(stc.lh_data)
    data_rh.append(stc.rh_data)

axs[0].plot(stc.times*1000, np.mean(np.mean(data_lh, axis=0), axis=0).T, color='#1f77b4')
axs[0].plot(stc.times*1000, np.mean(np.mean(data_rh, axis=0), axis=0).T, color='#ff7f0e')
axs[0].grid(axis='x'); axs[0].set_ylim([0, ymax])

data_lh = []
data_rh = []
for stc_file in tqdm(stc_files_list_2):
    stc = mne.read_source_estimate(fname=stc_file, subject='fsaverage')
    data_lh.append(stc.lh_data)
    data_rh.append(stc.rh_data)

axs[1].plot(stc.times*1000, np.mean(np.mean(data_lh, axis=0), axis=0).T, color='#1f77b4')
axs[1].plot(stc.times*1000, np.mean(np.mean(data_rh, axis=0), axis=0).T, color='#ff7f0e')
axs[1].grid(axis='x'); axs[1].set_ylim([0, ymax])

data_lh = []
data_rh = []
for stc_file in tqdm(stc_files_list_3):
    stc = mne.read_source_estimate(fname=stc_file, subject='fsaverage')
    data_lh.append(stc.lh_data)
    data_rh.append(stc.rh_data)

axs[2].plot(stc.times*1000, np.mean(np.mean(data_lh, axis=0), axis=0).T, color='#1f77b4')
axs[2].plot(stc.times*1000, np.mean(np.mean(data_rh, axis=0), axis=0).T, color='#ff7f0e')
axs[2].grid(axis='x'); axs[2].set_ylim([0, ymax])

data_lh = []
data_rh = []
for stc_file in tqdm(stc_files_list_4):
    stc = mne.read_source_estimate(fname=stc_file, subject='fsaverage')
    data_lh.append(stc.lh_data)
    data_rh.append(stc.rh_data)

axs[3].plot(stc.times*1000, np.mean(np.mean(data_lh, axis=0), axis=0).T, color='#1f77b4')
axs[3].plot(stc.times*1000, np.mean(np.mean(data_rh, axis=0), axis=0).T, color='#ff7f0e')
axs[3].grid(axis='x'); axs[3].set_ylim([0, ymax])

data_lh = []
data_rh = []
for stc_file in tqdm(stc_files_list_5):
    stc = mne.read_source_estimate(fname=stc_file, subject='fsaverage')
    data_lh.append(stc.lh_data)
    data_rh.append(stc.rh_data)

axs[4].plot(stc.times*1000, np.mean(np.mean(data_lh, axis=0), axis=0).T, color='#1f77b4')
axs[4].plot(stc.times*1000, np.mean(np.mean(data_rh, axis=0), axis=0).T, color='#ff7f0e')
axs[4].grid(axis='x'); axs[4].set_ylim([0, ymax])

data_lh = []
data_rh = []
for stc_file in tqdm(stc_files_list_6):
    stc = mne.read_source_estimate(fname=stc_file, subject='fsaverage')
    data_lh.append(stc.lh_data)
    data_rh.append(stc.rh_data)

axs[5].plot(stc.times*1000, np.mean(np.mean(data_lh, axis=0), axis=0).T, color='#1f77b4')
axs[5].plot(stc.times*1000, np.mean(np.mean(data_rh, axis=0), axis=0).T, color='#ff7f0e')
axs[5].grid(axis='x'); axs[5].set_ylim([0, ymax])

Plotting all subjects * conditions

In [ ]:
subject = '853'
directory = '/Users/payamsadeghishabestari/KI_MEG/stcs/tinmeg1_morphed'
stc_files_list = []
for filename in sorted(os.listdir(directory)): 
    f = os.path.join(directory, filename)
    # checking if it is a file
    if os.path.isfile(f) and f.endswith("-lh.stc") and (subject in f): # or -rh
        stc_files_list.append(f)

event1 = 'PO60_90'
event2 = 'GO_60'
event3 = 'GP60_i0'
event4 = 'GP60_i60'
event5 = 'GP60_i120'
event6 = 'GP60_i240'
fig, axs = plt.subplots(1, 6, figsize=(17, 2))
ymax = 5
for stc_file in stc_files_list:
    if event1 in stc_file:
        stc = mne.read_source_estimate(fname=stc_file, subject='fsaverage')
        axs[0].plot(stc.times*1000, np.mean(stc.lh_data, axis=0).T, color='#1f77b4')
        axs[0].plot(stc.times*1000, np.mean(stc.rh_data, axis=0).T, color='#ff7f0e')
        axs[0].grid(axis='x'); axs[0].set_ylim([0, ymax])
    if event2 in stc_file:
        stc = mne.read_source_estimate(fname=stc_file, subject='fsaverage')
        axs[1].plot(stc.times*1000, np.mean(stc.lh_data, axis=0).T, color='#1f77b4')
        axs[1].plot(stc.times*1000, np.mean(stc.rh_data, axis=0).T, color='#ff7f0e')
        axs[1].grid(axis='x'); axs[1].set_ylim([0, ymax])
    if event3 in stc_file:
        stc = mne.read_source_estimate(fname=stc_file, subject='fsaverage')
        axs[2].plot(stc.times*1000, np.mean(stc.lh_data, axis=0).T, color='#1f77b4')
        axs[2].plot(stc.times*1000, np.mean(stc.rh_data, axis=0).T, color='#ff7f0e')
        axs[2].grid(axis='x'); axs[2].set_ylim([0, ymax])
    if event4 in stc_file:
        stc = mne.read_source_estimate(fname=stc_file, subject='fsaverage')
        axs[3].plot(stc.times*1000, np.mean(stc.lh_data, axis=0).T, color='#1f77b4')
        axs[3].plot(stc.times*1000, np.mean(stc.rh_data, axis=0).T, color='#ff7f0e')
        axs[3].grid(axis='x'); axs[3].set_ylim([0, ymax])
    if event5 in stc_file:
        stc = mne.read_source_estimate(fname=stc_file, subject='fsaverage')
        axs[4].plot(stc.times*1000, np.mean(stc.lh_data, axis=0).T, color='#1f77b4')
        axs[4].plot(stc.times*1000, np.mean(stc.rh_data, axis=0).T, color='#ff7f0e')
        axs[4].grid(axis='x'); axs[4].set_ylim([0, ymax])
    if event6 in stc_file:
        stc = mne.read_source_estimate(fname=stc_file, subject='fsaverage')
        axs[5].plot(stc.times*1000, np.mean(stc.lh_data, axis=0).T, color='#1f77b4')
        axs[5].plot(stc.times*1000, np.mean(stc.rh_data, axis=0).T, color='#ff7f0e')
        axs[5].grid(axis='x'); axs[5].set_ylim([0, ymax])



Plotting time courses in transverstemporal

In [ ]:
subject = '863'
directory = '/Users/payamsadeghishabestari/KI_MEG/stcs/tinmeg1_morphed'
stc_files_list = []
for filename in sorted(os.listdir(directory)): 
    f = os.path.join(directory, filename)
    # checking if it is a file
    if os.path.isfile(f) and f.endswith("-lh.stc") and (subject in f): # or -rh
        stc_files_list.append(f)

event1 = 'PO60_90'
event2 = 'GO_60'
event3 = 'GP60_i0'
event4 = 'GP60_i60'
event5 = 'GP60_i120'
event6 = 'GP60_i240'

brain_labels = mne.read_labels_from_annot(subject='fsaverage', parc='aparc')[:-1] # 68 labels
fs_dir = fetch_fsaverage(verbose=False)
src_fname = op.join(fs_dir, "bem", "fsaverage-ico-5-src.fif")
src = mne.read_source_spaces(src_fname, verbose=False)
bl_idx_rh = -1
bl_idx_lh = -2
mode = 'mean'

fig, axs = plt.subplots(1, 6, figsize=(17, 2))
ymax = 12
for stc_file in stc_files_list:
    if event1 in stc_file:
        stc = mne.read_source_estimate(fname=stc_file, subject='fsaverage')
        stc_rh = stc.extract_label_time_course(brain_labels[bl_idx_rh], src, mode=mode, verbose=False)
        stc_lh = stc.extract_label_time_course(brain_labels[bl_idx_lh], src, mode=mode, verbose=False)
        axs[0].plot(stc.times*1000, stc_rh[0], color='#1f77b4')
        axs[0].plot(stc.times*1000, stc_lh[0], color='#ff7f0e')
        axs[0].grid(axis='x'); axs[0].set_ylim([0, ymax])
    if event2 in stc_file:
        stc = mne.read_source_estimate(fname=stc_file, subject='fsaverage')
        stc_rh = stc.extract_label_time_course(brain_labels[bl_idx_rh], src, mode=mode, verbose=False)
        stc_lh = stc.extract_label_time_course(brain_labels[bl_idx_lh], src, mode=mode, verbose=False)
        axs[1].plot(stc.times*1000, stc_rh[0], color='#1f77b4')
        axs[1].plot(stc.times*1000, stc_lh[0], color='#ff7f0e')
        axs[1].grid(axis='x'); axs[1].set_ylim([0, ymax])
    if event3 in stc_file:
        stc = mne.read_source_estimate(fname=stc_file, subject='fsaverage')
        stc_rh = stc.extract_label_time_course(brain_labels[bl_idx_rh], src, mode=mode, verbose=False)
        stc_lh = stc.extract_label_time_course(brain_labels[bl_idx_lh], src, mode=mode, verbose=False)
        axs[2].plot(stc.times*1000, stc_rh[0], color='#1f77b4')
        axs[2].plot(stc.times*1000, stc_lh[0], color='#ff7f0e')
        axs[2].grid(axis='x'); axs[2].set_ylim([0, ymax])
    if event4 in stc_file:
        stc = mne.read_source_estimate(fname=stc_file, subject='fsaverage')
        stc_rh = stc.extract_label_time_course(brain_labels[bl_idx_rh], src, mode=mode, verbose=False)
        stc_lh = stc.extract_label_time_course(brain_labels[bl_idx_lh], src, mode=mode, verbose=False)
        axs[3].plot(stc.times*1000, stc_rh[0], color='#1f77b4')
        axs[3].plot(stc.times*1000, stc_lh[0], color='#ff7f0e')
        axs[3].grid(axis='x'); axs[3].set_ylim([0, ymax])
    if event5 in stc_file:
        stc = mne.read_source_estimate(fname=stc_file, subject='fsaverage')
        stc_rh = stc.extract_label_time_course(brain_labels[bl_idx_rh], src, mode=mode, verbose=False)
        stc_lh = stc.extract_label_time_course(brain_labels[bl_idx_lh], src, mode=mode, verbose=False)
        axs[4].plot(stc.times*1000, stc_rh[0], color='#1f77b4')
        axs[4].plot(stc.times*1000, stc_lh[0], color='#ff7f0e')
        axs[4].grid(axis='x'); axs[4].set_ylim([0, ymax])
    if event6 in stc_file:
        stc = mne.read_source_estimate(fname=stc_file, subject='fsaverage')
        stc_rh = stc.extract_label_time_course(brain_labels[bl_idx_rh], src, mode=mode, verbose=False)
        stc_lh = stc.extract_label_time_course(brain_labels[bl_idx_lh], src, mode=mode, verbose=False)
        axs[5].plot(stc.times*1000, stc_rh[0], color='#1f77b4')
        axs[5].plot(stc.times*1000, stc_lh[0], color='#ff7f0e')
        axs[5].grid(axis='x'); axs[5].set_ylim([0, ymax])

In [ ]:
# average in transtemporal
subject = '863'
directory = '/Users/payamsadeghishabestari/KI_MEG/stcs/tinmeg1_morphed'
stc_files_list = []
for filename in sorted(os.listdir(directory)): 
    f = os.path.join(directory, filename)
    # checking if it is a file
    if os.path.isfile(f) and f.endswith("-lh.stc"): # or -rh
        stc_files_list.append(f)

event1 = 'PO60_90'
event2 = 'GO_60'
event3 = 'GP60_i0'
event4 = 'GP60_i60'
event5 = 'GP60_i120'
event6 = 'GP60_i240'

brain_labels = mne.read_labels_from_annot(subject='fsaverage', parc='aparc')[:-1] # 68 labels
fs_dir = fetch_fsaverage(verbose=False)
src_fname = op.join(fs_dir, "bem", "fsaverage-ico-5-src.fif")
src = mne.read_source_spaces(src_fname, verbose=False)
bl_idx_rh = -1
bl_idx_lh = -2
mode = 'mean'

fig, axs = plt.subplots(1, 6, figsize=(17, 2))
ymax = 12
stcs_rh_1 = []; stcs_lh_1 = []
stcs_rh_2 = []; stcs_lh_2 = []
stcs_rh_3 = []; stcs_lh_3 = []
stcs_rh_4 = []; stcs_lh_4 = []
stcs_rh_5 = []; stcs_lh_5 = []
stcs_rh_6 = []; stcs_lh_6 = []

for stc_file in stc_files_list:
    if event1 in stc_file:
        stc = mne.read_source_estimate(fname=stc_file, subject='fsaverage')
        stcs_rh_1.append(stc.extract_label_time_course(brain_labels[bl_idx_rh], src, mode=mode, verbose=False))
        stcs_lh_1.append(stc.extract_label_time_course(brain_labels[bl_idx_lh], src, mode=mode, verbose=False))
    if event2 in stc_file:
        stc = mne.read_source_estimate(fname=stc_file, subject='fsaverage')
        stcs_rh_2.append(stc.extract_label_time_course(brain_labels[bl_idx_rh], src, mode=mode, verbose=False))
        stcs_lh_2.append(stc.extract_label_time_course(brain_labels[bl_idx_lh], src, mode=mode, verbose=False))
    if event3 in stc_file:
        stc = mne.read_source_estimate(fname=stc_file, subject='fsaverage')
        stcs_rh_3.append(stc.extract_label_time_course(brain_labels[bl_idx_rh], src, mode=mode, verbose=False))
        stcs_lh_3.append(stc.extract_label_time_course(brain_labels[bl_idx_lh], src, mode=mode, verbose=False))
    if event4 in stc_file:
        stc = mne.read_source_estimate(fname=stc_file, subject='fsaverage')
        stcs_rh_4.append(stc.extract_label_time_course(brain_labels[bl_idx_rh], src, mode=mode, verbose=False))
        stcs_lh_4.append(stc.extract_label_time_course(brain_labels[bl_idx_lh], src, mode=mode, verbose=False))
    if event5 in stc_file:
        stc = mne.read_source_estimate(fname=stc_file, subject='fsaverage')
        stcs_rh_5.append(stc.extract_label_time_course(brain_labels[bl_idx_rh], src, mode=mode, verbose=False))
        stcs_lh_5.append(stc.extract_label_time_course(brain_labels[bl_idx_lh], src, mode=mode, verbose=False))
    if event6 in stc_file:
        stc = mne.read_source_estimate(fname=stc_file, subject='fsaverage')
        stcs_rh_6.append(stc.extract_label_time_course(brain_labels[bl_idx_rh], src, mode=mode, verbose=False))
        stcs_lh_6.append(stc.extract_label_time_course(brain_labels[bl_idx_lh], src, mode=mode, verbose=False))


axs[0].plot(stc.times*1000, np.mean(np.array(stcs_rh_1),axis=0)[0], color='#1f77b4')
axs[0].plot(stc.times*1000, np.mean(np.array(stcs_lh_1),axis=0)[0], color='#ff7f0e')
axs[0].grid(axis='x'); axs[0].set_ylim([0, ymax])
axs[1].plot(stc.times*1000, np.mean(np.array(stcs_rh_2),axis=0)[0], color='#1f77b4')
axs[1].plot(stc.times*1000, np.mean(np.array(stcs_lh_2),axis=0)[0], color='#ff7f0e')
axs[1].grid(axis='x'); axs[1].set_ylim([0, ymax])
axs[2].plot(stc.times*1000, np.mean(np.array(stcs_rh_3),axis=0)[0], color='#1f77b4')
axs[2].plot(stc.times*1000, np.mean(np.array(stcs_lh_3),axis=0)[0], color='#ff7f0e')
axs[2].grid(axis='x'); axs[2].set_ylim([0, ymax])
axs[3].plot(stc.times*1000, np.mean(np.array(stcs_rh_4),axis=0)[0], color='#1f77b4')
axs[3].plot(stc.times*1000, np.mean(np.array(stcs_lh_4),axis=0)[0], color='#ff7f0e')
axs[3].grid(axis='x'); axs[3].set_ylim([0, ymax])
axs[4].plot(stc.times*1000, np.mean(np.array(stcs_rh_5),axis=0)[0], color='#1f77b4')
axs[4].plot(stc.times*1000, np.mean(np.array(stcs_lh_5),axis=0)[0], color='#ff7f0e')
axs[4].grid(axis='x'); axs[4].set_ylim([0, ymax])
axs[5].plot(stc.times*1000, np.mean(np.array(stcs_rh_6),axis=0)[0], color='#1f77b4')
axs[5].plot(stc.times*1000, np.mean(np.array(stcs_lh_6),axis=0)[0], color='#ff7f0e')
axs[5].grid(axis='x'); axs[5].set_ylim([0, ymax])

Statistics

In [ ]:
# test if the evoked response is significantly different between two conditions across subjects
event_1 = 'PO70_95'
event_2 = 'GP70_i240'

# iterate over files, reading files and check
directory = '/Users/payamsadeghishabestari/KI_MEG/stcs/tinmeg1_morphed'
stc_files_list_ev_1 = []
stc_files_list_ev_2 = []
for filename in sorted(os.listdir(directory)): 
    f = os.path.join(directory, filename)
    if os.path.isfile(f) and f.endswith("-lh.stc"): # or -rh
        if event_1 in f:
            stc_files_list_ev_1.append(f)
        if event_2 in f:
            stc_files_list_ev_2.append(f)

if len(stc_files_list_ev_1) == len(stc_files_list_ev_2):
    n_subjects = len(stc_files_list_ev_2)
else:
    raise ValueError('number of subjects for two events are not same')

data_ev_1 = []
data_ev_2 = []
for stc_file_ev_1, stc_file_ev_2 in zip(stc_files_list_ev_1, stc_files_list_ev_2):
    stc_ev1 = mne.read_source_estimate(fname=stc_file_ev_1, subject='fsaverage')
    stc_ev2 = mne.read_source_estimate(fname=stc_file_ev_2, subject='fsaverage')
    data_ev_1.append(stc_ev1.data)
    data_ev_2.append(stc_ev2.data)

# Creating the big matrix
n_vertices, n_times = stc_ev1.data.shape
tmin = stc_ev1.tmin
tstep = stc_ev1.tstep * 1000 
np.random.seed(0)
X = np.zeros((n_vertices, n_times, n_subjects, 2))

for condition_idx, condition in enumerate(data_ev_1):
    X[:, :, condition_idx, 0] += condition
for condition_idx, condition in enumerate(data_ev_2):
    X[:, :, condition_idx, 1] += condition

X = np.abs(X)  
X = X[:, :, :, 0] - X[:, :, :, 1] 
X = np.transpose(X, [2, 1, 0]) # observations (subjects) × time × space

# compute adjacency matrix
src_fname = '/Users/payamsadeghishabestari/mne_data/MNE-fsaverage-data/fsaverage/bem/fsaverage-ico-5-src.fif'
src = mne.read_source_spaces(src_fname, verbose=False)
fsave_vertices = [s["vertno"] for s in src]
adjacency = mne.spatial_src_adjacency(src, verbose=False)

# cluster forming threshold based on a p-value (two-tailed test)
p_threshold = 0.001
df = n_subjects - 1  
n_permutations = 1024
t_threshold = stats.distributions.t.ppf(1 - p_threshold / 2, df=df)
T_obs, clusters, cluster_p_values, H0 = clu = spatio_temporal_cluster_1samp_test(
                                                X, threshold=t_threshold,
                                                n_permutations=n_permutations, adjacency=adjacency,
                                                n_jobs=-1, verbose=True)

# Select the clusters that are statistically significant at p < 0.05
good_clusters_idx = np.where(cluster_p_values < 0.05)[0]
good_clusters = [clusters[idx] for idx in good_clusters_idx]

# Visualize the clusters (blue blobs are for event 1 < event 2, red for event 1 > event 2)
colormap = "auto"
clim = dict(kind="value", lims=[0, 1, 40])
stc_all_cluster_vis = summarize_clusters_stc(clu, tstep=tstep, subject="fsaverage", vertices=fsave_vertices)
brain = stc_all_cluster_vis.plot(views="lat", hemi="split", size=(800, 400), subject='fsaverage',
                    subjects_dir=None, background="w", colorbar=True, clim=clim, colormap=colormap, 
                    time_label="temporal extent (ms)", time_viewer=True, show_traces=True, smoothing_steps=4)

# ***** The first time point in this SourceEstimate object is the summation of all the clusters.
#       Subsequent time points contain each individual cluster.
#       The magnitude of the activity corresponds to the duration spanned by the cluster *****

from Source to Label

In [ ]:
# checking only one example (also checking different modes)
stc_fname = '/Users/payamsadeghishabestari/KI_MEG/stcs/tinmeg1_morphed/539_PO60_90-lh.stc-lh.stc'
brain_labels = mne.read_labels_from_annot(subject='fsaverage', parc='aparc')[:-1] # 68 labels
fs_dir = fetch_fsaverage(verbose=False)
src_fname = op.join(fs_dir, "bem", "fsaverage-ico-5-src.fif")
src = mne.read_source_spaces(src_fname, verbose=False)
bl_idx = -1
# extracting time course from source_estimate object in label index 0 (bankssts-lh)
stc = mne.read_source_estimate(fname=stc_fname, subject='fsaverage')
stc_v_label = stc.in_label(brain_labels[bl_idx])
modes = ("mean", "mean_flip", "pca_flip", "max")
tcs = dict()
for mode in modes:
    tcs[mode] = stc.extract_label_time_course(brain_labels[bl_idx], src, mode=mode, verbose=False)

# plotting
fig, ax = plt.subplots(1,1, figsize=(11,5))
t = 1e3 * stc.times
ax.plot(t, stc_v_label.data.T, "k", linewidth=0.5, alpha=0.2)
for mode, tc in tcs.items():
    ax.plot(t, tc[0], linewidth=3, label=str(mode))

ax.legend(loc="upper right")
ax.set(xlabel="Time (ms)", ylabel="Source amplitude",
    title="Activations in Label %r" % (brain_labels[bl_idx].name))
mne.viz.tight_layout()

In [ ]:
# loading all sibjects
directory = '/Users/payamsadeghishabestari/KI_MEG/stcs/tinmeg1_morphed'
stc_files_list = []
event = 'GO_70'
for filename in sorted(os.listdir(directory)): 
    f = os.path.join(directory, filename)
    if os.path.isfile(f) and f.endswith("-lh.stc") and event in f: # or -rh
            stc_files_list.append(f)

# initializing
brain_labels = mne.read_labels_from_annot(subject='fsaverage', parc='aparc')[:-1] # 68 labels
fs_dir = fetch_fsaverage(verbose=False)
src_fname = op.join(fs_dir, "bem", "fsaverage-ico-5-src.fif")
src = mne.read_source_spaces(src_fname, verbose=False)
rh_labels = [bl.name for bl in brain_labels if '-rh' in bl.name]
lh_labels = [bl.name for bl in brain_labels if '-lh' in bl.name]
dict_max_rh_label = {} 
dict_max_lh_label = {} 

for stc_fname in tqdm(stc_files_list):
    subject_id = stc_fname[58:61]
    stc = mne.read_source_estimate(fname=stc_fname, subject='fsaverage')
    
    tcs_max_rh = []
    tcs_max_lh = []
    for bl_idx in range(len(brain_labels)):
        if '-rh' in brain_labels[bl_idx].name:
            tcs_max_rh.append(stc.extract_label_time_course(brain_labels[bl_idx], src, mode='mean', verbose=False).max())
        if '-lh' in brain_labels[bl_idx].name:
            tcs_max_lh.append(stc.extract_label_time_course(brain_labels[bl_idx], src, mode='mean', verbose=False).max())

    bl_rh_max = rh_labels[np.argmax(np.array(tcs_max_rh))]
    bl_lh_max = lh_labels[np.argmax(np.array(tcs_max_lh))]

    dict_max_rh_label[subject_id] = bl_rh_max
    dict_max_lh_label[subject_id] = bl_lh_max

Making big dataframe with all information

In [ ]:
# making big dataframe for transverstemporal rh/lh
my_dict = {'subject ID': [], 'Stimulus': [], 'Hemisphere': [], 'latency': [],
        'PSNR': [], 'peak value (30ms)': [], 'peak value (100ms)': [], 'area value (30ms)': [],
        'area value (100ms)': [], 'amplitude inhibition (30ms)': [], 'area inhibition (30ms)': [],
        'amplitude inhibition (100ms)': [], 'area inhibition (100ms)': [], 'EOG peak': [],
        'EOG ptp': [], 'EOG area (30ms)': [], 'EOG area (100ms)': [], 'EOG peak latency': []} 

# events = ['PO60_70', 'PO60_75', 'PO60_80', 'PO60_85', 'PO60_90', 'PO60_95',
#         'PO70_75', 'PO70_80', 'PO70_85', 'PO70_90', 'PO70_95',
#         'GP60_i0', 'GP60_i60', 'GP60_i120', 'GP60_i240',
#         'GP70_i0', 'GP70_i60', 'GP70_i120', 'GP70_i240',
#         'GO_60', 'GO_70']

events = ['GPP_00', 'GPG_00', 'PO_00', 'GO_00', 'PPP_00', 'PPG_00',
        'GPP_03', 'GPG_03', 'PO_03', 'GO_03',
        'GPP_08', 'GPG_08', 'PO_08', 'GO_08',
        'GPP_30', 'GPG_30', 'PO_30', 'GO_30',
        'GPP_33', 'GPG_33', 'PO_33', 'GO_33',
        'GPP_38', 'GPG_38', 'PO_38', 'GO_38',
        'GPP_80', 'GPG_80', 'PO_80', 'GO_80',
        'GPP_83', 'GPG_83', 'PO_83', 'GO_83',
        'GPP_88', 'GPG_88', 'PO_88', 'GO_88']

# events = ['GPP_00', 'GPG_00', 'PO_00', 'GO_00', 'PPP_00', 'PPG_00',
#         'GPP_03', 'GPG_03', 'PO_03', 'GO_03',
#         'GPP_08', 'GPG_08', 'PO_08', 'GO_08']

# subjects = ['539', '697', '750', '756', '832', '835', '836', '838', '839',
#             '840', '841', '842', '844', '845', '847', '849', '850', '852', '853',
#             '856', '857', '858', '859', '861', '862', '863'] # should I exclude 3 subjects?

subjects = ['916', '979', '980', '981', '982', '983', '984', '986', '988']

# subjects = ['1004', '1006', '1008', '1009', '1017', '1021', '1025', '1031',
#             '1032', '1033', '1034', '1035', '1037', '1038', '1044', '1045', '1047', '1048']

brain_labels = mne.read_labels_from_annot(subject='fsaverage', parc='aparc')[:-1] # 68 labels
fs_dir = fetch_fsaverage(verbose=False)
src_fname = op.join(fs_dir, "bem", "fsaverage-ico-5-src.fif")
src = mne.read_source_spaces(src_fname, verbose=False)
directory_stcs = '/Users/payamsadeghishabestari/KI_MEG/stcs/tinmeg2_morphed'
directory_eps = '/Users/payamsadeghishabestari/KI_MEG/epochs/tinmeg2/epochs_bads_dropped'

# looping over all files
for subject in tqdm(subjects):
    for stim in events:
        for hemi in ['lh', 'rh']:
            for filename in sorted(os.listdir(directory_stcs)): 
                f = os.path.join(directory_stcs, filename)
                if os.path.isfile(f) and f.endswith(f"-{hemi}.stc") and stim in f and subject in f:
                    
                    # reading source estimate file
                    stc_fname = f
                    stc = mne.read_source_estimate(fname=stc_fname, subject='fsaverage')
                    
                    # reading epoch file for eog response
                    for filename in sorted(os.listdir(directory_eps)): 
                        f = os.path.join(directory_eps, filename)
                        if os.path.isfile(f) and f.endswith('-epo.fif') and subject in f:
                            ep_fname = f
                            ep = mne.read_epochs(fname=ep_fname, preload=True, verbose=False)
                    
                    eog_peak_value = abs(np.squeeze(ep[stim].get_data(picks=['EOG002'])).mean(axis=0)[75:].min() * 1e6) # only positive
                    argmin = np.squeeze(ep[stim].get_data(picks=['EOG002'])).mean(axis=0)[75:].argmin() + 75
                    eog_peak_time = np.linspace(-300, 300, 151)[argmin]
                    (t1, t2) = (argmin - 5, argmin + 4)
                    (t3, t4) = (argmin - 14, argmin + 13)
                    my_dict['EOG peak'].append(eog_peak_value)
                    my_dict['EOG ptp'].append(np.ptp(np.squeeze(ep[stim].get_data(picks=['EOG002'])).mean(axis=0)[75:]) * 1e6)
                    my_dict['EOG area (30ms)'].append(abs(np.squeeze(ep[stim].get_data(picks=['EOG002'])).mean(axis=0)[t1:t2].sum() * 1e6))
                    my_dict['EOG area (100ms)'].append(abs(np.squeeze(ep[stim].get_data(picks=['EOG002'])).mean(axis=0)[t3:t4].sum() * 1e6))
                    my_dict['EOG peak latency'].append(eog_peak_time)
                    
                    # localizing the brain label
                    if hemi == 'rh':
                        bl_idx = -1 # transversetemporal-rh
                    if hemi == 'lh':
                        bl_idx = -2 # transversetemporal-lh

                    # computing some params in stc object   
                    tcs_noise_avg = stc.extract_label_time_course(brain_labels[bl_idx], src, mode='mean', verbose=False)[:,:76].mean()
                    tcs_peak_100ms = stc.extract_label_time_course(brain_labels[bl_idx], src, mode='mean', verbose=False)[:,87:114].max()
                    tcs_peak_30ms = stc.extract_label_time_course(brain_labels[bl_idx], src, mode='mean', verbose=False)[:,96:105].max()
                    tcs_area_100ms = stc.extract_label_time_course(brain_labels[bl_idx], src, mode='mean', verbose=False)[:,87:114].sum()
                    tcs_area_30ms = stc.extract_label_time_course(brain_labels[bl_idx], src, mode='mean', verbose=False)[:,96:105].sum()
                    
                    # computing inhibition indexes for tinmeg1
                    # if '60' in stim and '70' not in stim:
                    #     stc_fname_stn = f'/Users/payamsadeghishabestari/KI_MEG/stcs/tinmeg1_morphed/{subject}_PO60_90-lh.stc-lh.stc'
                    #     stc_stn = mne.read_source_estimate(fname=stc_fname_stn, subject='fsaverage')
                    #     tcs_peak_30ms_stn = stc_stn.extract_label_time_course(brain_labels[bl_idx],
                    #                                                             src, mode='mean', verbose=False)[:,96:105].max()
                    #     tcs_area_30ms_stn = stc_stn.extract_label_time_course(brain_labels[bl_idx], src, mode='mean', verbose=False)[:,96:105].sum()
                    #     tcs_peak_100ms_stn = stc_stn.extract_label_time_course(brain_labels[bl_idx],
                    #                                                             src, mode='mean', verbose=False)[:,87:114].max()
                    #     tcs_area_100ms_stn = stc_stn.extract_label_time_course(brain_labels[bl_idx], src, mode='mean', verbose=False)[:,87:114].sum()

                    # if '70' in stim:
                    #     stc_fname_stn = f'/Users/payamsadeghishabestari/KI_MEG/stcs/tinmeg1_morphed/{subject}_PO70_90-lh.stc-lh.stc'
                    #     stc_stn = mne.read_source_estimate(fname=stc_fname_stn, subject='fsaverage')
                    #     tcs_peak_30ms_stn = stc_stn.extract_label_time_course(brain_labels[bl_idx],
                    #                                                             src, mode='mean', verbose=False)[:,96:105].max()
                    #     tcs_area_30ms_stn = stc_stn.extract_label_time_course(brain_labels[bl_idx], src, mode='mean', verbose=False)[:,96:105].sum()
                    #     tcs_peak_100ms_stn = stc_stn.extract_label_time_course(brain_labels[bl_idx],
                    #                                                             src, mode='mean', verbose=False)[:,87:114].max()
                    #     tcs_area_100ms_stn = stc_stn.extract_label_time_course(brain_labels[bl_idx], src, mode='mean', verbose=False)[:,87:114].sum()
            
                    # computing inhibition indexes for tinmeg2
                    stc_fname_stn = f'/Users/payamsadeghishabestari/KI_MEG/stcs/tinmeg2_morphed/{subject}_PO_{stim[-2:]}-lh.stc-lh.stc'
                    stc_stn = mne.read_source_estimate(fname=stc_fname_stn, subject='fsaverage')
                    tcs_peak_30ms_stn = stc_stn.extract_label_time_course(brain_labels[bl_idx],
                                                                            src, mode='mean', verbose=False)[:,96:105].max()
                    tcs_area_30ms_stn = stc_stn.extract_label_time_course(brain_labels[bl_idx], src, mode='mean', verbose=False)[:,96:105].sum()
                    tcs_peak_100ms_stn = stc_stn.extract_label_time_course(brain_labels[bl_idx],
                                                                            src, mode='mean', verbose=False)[:,87:114].max()
                    tcs_area_100ms_stn = stc_stn.extract_label_time_course(brain_labels[bl_idx], src, mode='mean', verbose=False)[:,87:114].sum()
                    
                    
                    
                    my_dict['amplitude inhibition (30ms)'].append((1 - (tcs_peak_30ms / tcs_peak_30ms_stn)) * 100)
                    my_dict['area inhibition (30ms)'].append((1 - (tcs_area_30ms / tcs_area_30ms_stn)) * 100)
                    my_dict['amplitude inhibition (100ms)'].append((1 - (tcs_peak_100ms / tcs_peak_100ms_stn)) * 100)
                    my_dict['area inhibition (100ms)'].append((1 - (tcs_area_100ms / tcs_area_100ms_stn)) * 100)

                    # putting in the dictionary
                    my_dict['subject ID'].append(subject)
                    my_dict['Stimulus'].append(stim)
                    my_dict['Hemisphere'].append(hemi)
                    my_dict['latency'].append(stc.extract_label_time_course(brain_labels[bl_idx], src,
                                                                            mode='mean', verbose=False)[:,87:114].argmax() + 87) 
                    my_dict['PSNR'].append(20 * np.log10(tcs_peak_30ms / tcs_noise_avg))
                    my_dict['peak value (30ms)'].append(tcs_peak_30ms)
                    my_dict['peak value (100ms)'].append(tcs_peak_100ms)
                    my_dict['area value (30ms)'].append(tcs_area_30ms)
                    my_dict['area value (100ms)'].append(tcs_area_100ms)
                    
df = pd.DataFrame(my_dict)
# save it
df.to_csv('/Users/payamsadeghishabestari/KI_MEG/dataframes/tinmeg2_transverstemporal.csv')

In [ ]:
# making big dataframe total left and right hemispheres
my_dict = {'subject ID': [], 'Stimulus': [], 'Hemisphere': [], 'latency': [],
        'PSNR': [], 'peak value (30ms)': [], 'peak value (100ms)': [], 'area value (30ms)': [],
        'area value (100ms)': [], 'amplitude inhibition (30ms)': [], 'area inhibition (30ms)': [],
        'amplitude inhibition (100ms)': [], 'area inhibition (100ms)': [], 'EOG peak': [],
        'EOG ptp': [], 'EOG area (30ms)': [], 'EOG area (100ms)': []}

events = ['PO60_70', 'PO60_75', 'PO60_80', 'PO60_85', 'PO60_90', 'PO60_95',
        'PO70_75', 'PO70_80', 'PO70_85', 'PO70_90', 'PO70_95',
        'GP60_i0', 'GP60_i60', 'GP60_i120', 'GP60_i240',
        'GP70_i0', 'GP70_i60', 'GP70_i120', 'GP70_i240',
        'GO_60', 'GO_70']

subjects = ['539', '697', '750', '756', '832', '835', '836', '838', '839',
            '840', '841', '842', '844', '845', '847', '849', '850', '852', '853',
            '856', '857', '858', '859', '861', '862', '863'] 

brain_labels = mne.read_labels_from_annot(subject='fsaverage', parc='aparc')[:-1] # 68 labels
fs_dir = fetch_fsaverage(verbose=False)
src_fname = op.join(fs_dir, "bem", "fsaverage-ico-5-src.fif")
src = mne.read_source_spaces(src_fname, verbose=False)
directory_stcs = '/Users/payamsadeghishabestari/KI_MEG/stcs/tinmeg1_morphed'
directory_eps = '/Users/payamsadeghishabestari/KI_MEG/epochs/tinmeg1'

# looping over all files
for subject in tqdm(subjects):
    for stim in events:
        for hemi in ['lh', 'rh']:
            for filename in sorted(os.listdir(directory_stcs)): 
                f = os.path.join(directory_stcs, filename)
                if os.path.isfile(f) and f.endswith(f"-{hemi}.stc") and stim in f and subject in f:
                    
                    # reading source estimate file
                    stc_fname = f
                    stc = mne.read_source_estimate(fname=stc_fname, subject='fsaverage')
                    
                    # reading epoch file for eog response
                    for filename in sorted(os.listdir(directory_eps)): 
                        f = os.path.join(directory_eps, filename)
                        if os.path.isfile(f) and f.endswith('-epo.fif') and subject in f:
                            ep_fname = f
                            ep = mne.read_epochs(fname=ep_fname, preload=True, verbose=False)
                    
                    my_dict['EOG peak'].append(abs(np.squeeze(ep[stim].get_data(picks=['EOG002'])).mean(axis=0).min() * 1e6))
                    my_dict['EOG ptp'].append(np.ptp(np.squeeze(ep[stim].get_data(picks=['EOG002'])).mean(axis=0)) * 1e6)
                    my_dict['EOG area (30ms)'].append(abs(np.squeeze(ep[stim].get_data(picks=['EOG002'])).mean(axis=0)[96:105].sum() * 1e6))
                    my_dict['EOG area (100ms)'].append(abs(np.squeeze(ep[stim].get_data(picks=['EOG002'])).mean(axis=0)[87:114].sum() * 1e6))
                    
                    # localizing the brain label
                    if hemi == 'rh':
                        tcs_noise_avg = stc.rh_data.mean(axis=0)[:76].mean()
                        tcs_peak_100ms = stc.rh_data.mean(axis=0)[87:114].max()
                        tcs_peak_30ms = stc.rh_data.mean(axis=0)[96:105].max()
                        tcs_area_100ms = stc.rh_data.mean(axis=0)[87:114].sum()
                        tcs_area_30ms = stc.rh_data.mean(axis=0)[96:105].sum()
                        check = 1

                    if hemi == 'lh':
                        tcs_noise_avg = stc.lh_data.mean(axis=0)[:76].mean()
                        tcs_peak_100ms = stc.lh_data.mean(axis=0)[87:114].max()
                        tcs_peak_30ms = stc.lh_data.mean(axis=0)[96:105].max()
                        tcs_area_100ms = stc.lh_data.mean(axis=0)[87:114].sum()
                        tcs_area_30ms = stc.lh_data.mean(axis=0)[96:105].sum()
                        check = 2

                    # computing inhibition indexes
                    if '60' in stim and '70' not in stim and check==1:
                        stc_fname_stn = f'/Users/payamsadeghishabestari/KI_MEG/stcs/tinmeg1_morphed/{subject}_PO60_90-lh.stc-lh.stc'
                        stc_stn = mne.read_source_estimate(fname=stc_fname_stn, subject='fsaverage')
                        tcs_peak_30ms_stn = stc_stn.rh_data.mean(axis=0)[96:105].max()
                        tcs_area_30ms_stn = stc_stn.rh_data.mean(axis=0)[96:105].sum()
                        tcs_peak_100ms_stn = stc_stn.rh_data.mean(axis=0)[87:114].max()
                        tcs_area_100ms_stn = stc_stn.rh_data.mean(axis=0)[87:114].sum()

                    if '60' in stim and '70' not in stim and check==2:
                        stc_fname_stn = f'/Users/payamsadeghishabestari/KI_MEG/stcs/tinmeg1_morphed/{subject}_PO60_90-lh.stc-lh.stc'
                        stc_stn = mne.read_source_estimate(fname=stc_fname_stn, subject='fsaverage')
                        tcs_peak_30ms_stn = stc_stn.lh_data.mean(axis=0)[96:105].max()
                        tcs_area_30ms_stn = stc_stn.lh_data.mean(axis=0)[96:105].sum()
                        tcs_peak_100ms_stn = stc_stn.lh_data.mean(axis=0)[87:114].max()
                        tcs_area_100ms_stn = stc_stn.lh_data.mean(axis=0)[87:114].sum()

                    if '70' in stim and check==1:
                        stc_fname_stn = f'/Users/payamsadeghishabestari/KI_MEG/stcs/tinmeg1_morphed/{subject}_PO70_90-lh.stc-lh.stc'
                        stc_stn = mne.read_source_estimate(fname=stc_fname_stn, subject='fsaverage')
                        tcs_peak_30ms_stn = stc_stn.rh_data.mean(axis=0)[96:105].max()
                        tcs_area_30ms_stn = stc_stn.rh_data.mean(axis=0)[96:105].sum()
                        tcs_peak_100ms_stn = stc_stn.rh_data.mean(axis=0)[87:114].max()
                        tcs_area_100ms_stn = stc_stn.rh_data.mean(axis=0)[87:114].sum()

                    if '70' in stim and check==2:
                        stc_fname_stn = f'/Users/payamsadeghishabestari/KI_MEG/stcs/tinmeg1_morphed/{subject}_PO70_90-lh.stc-lh.stc'
                        stc_stn = mne.read_source_estimate(fname=stc_fname_stn, subject='fsaverage')
                        tcs_peak_30ms_stn = stc_stn.lh_data.mean(axis=0)[96:105].max()
                        tcs_area_30ms_stn = stc_stn.lh_data.mean(axis=0)[96:105].sum()
                        tcs_peak_100ms_stn = stc_stn.lh_data.mean(axis=0)[87:114].max()
                        tcs_area_100ms_stn = stc_stn.lh_data.mean(axis=0)[87:114].sum()
            
                    
                    my_dict['amplitude inhibition (30ms)'].append((1 - (tcs_peak_30ms / tcs_peak_30ms_stn)) * 100)
                    my_dict['area inhibition (30ms)'].append((1 - (tcs_area_30ms / tcs_area_30ms_stn)) * 100)
                    my_dict['amplitude inhibition (100ms)'].append((1 - (tcs_peak_100ms / tcs_peak_100ms_stn)) * 100)
                    my_dict['area inhibition (100ms)'].append((1 - (tcs_area_100ms / tcs_area_100ms_stn)) * 100)

                    if check==1:
                        my_dict['latency'].append(stc.rh_data.mean(axis=0)[87:114].argmax() + 87)
                    if check==2:
                        my_dict['latency'].append(stc.lh_data.mean(axis=0)[87:114].argmax() + 87)     
                    
                    # putting in the dictionary
                    my_dict['subject ID'].append(subject)
                    my_dict['Stimulus'].append(stim)
                    my_dict['Hemisphere'].append(hemi)
                    my_dict['PSNR'].append(20 * np.log10(tcs_peak_30ms / tcs_noise_avg))
                    my_dict['peak value (30ms)'].append(tcs_peak_30ms)
                    my_dict['peak value (100ms)'].append(tcs_peak_100ms)
                    my_dict['area value (30ms)'].append(tcs_area_30ms)
                    my_dict['area value (100ms)'].append(tcs_area_100ms)
                    
df = pd.DataFrame(my_dict)
# save it
df.to_csv('/Users/payamsadeghishabestari/KI_MEG/dataframe_tinmeg1_hemisphere.csv')

Plotting tinmeg1

In [ ]:
# load dataframe and remove three subjects
df = pd.read_csv('/Users/payamsadeghishabestari/KI_MEG/dataframes/tinmeg1_transverstemporal.csv') 
mask = df['subject ID'].isin([697, 750, 853, 841])
df = df[~mask]

In [ ]:
# dont run this part (it is alread applied)
# adding an extra column to the dataframe (eog inhibition)
subject_ids = np.array(df['subject ID']).astype(str)
stimuli = np.array(df['Stimulus']).astype(str)
eog_peaks = np.array(df['EOG peak'])
eog_ptps = np.array(df['EOG ptp'])
eog_areas = np.array(df['EOG area (30ms)'])

eog_amp_inhibits = []
eog_ptp_inhibits = []
eog_area_inhibits = []
for idx, (eog_peak, eog_ptp, eog_area) in enumerate(zip(eog_peaks, eog_ptps, eog_areas)):
    sub_id = subject_ids[idx]
    stimulus = stimuli[idx]
    array1 = np.where(subject_ids == sub_id)[0]
    
    if '60' in stimulus:
        array2 = np.where(stimuli == 'PO60_90')[0]
    if '70' in stimulus:
        array2 = np.where(stimuli == 'PO70_90')[0]
    
    set1 = set(array1)
    set2 = set(array2)
    common_idx = list(set1.intersection(set2))[0]
    eog_peak_stn = eog_peaks[common_idx]
    eog_ptp_stn = eog_ptps[common_idx]
    eog_area_stn = eog_areas[common_idx]
    
    eog_amp_inhibit = (1 - eog_peak / eog_peak_stn) * 100
    eog_ptp_inhibit = (1 - eog_ptp / eog_ptp_stn) * 100
    eog_area_inhibit = (1 - eog_area / eog_area_stn) * 100
    
    eog_amp_inhibits.append(eog_amp_inhibit)
    eog_ptp_inhibits.append(eog_ptp_inhibit)
    eog_area_inhibits.append(eog_area_inhibit)

df['EOG peak inhibition'] = eog_amp_inhibits
df['EOG ptp inhibition'] = eog_ptp_inhibits
df['EOG area inhibition'] = eog_area_inhibits

Figure 1

In [ ]:
# plotting Figure 1 a
fig, ax = plt.subplots(1, 1, figsize=(4,4))
color='grey'
order_1 = ['PO60_70', 'PO60_75', 'PO60_80', 'PO60_85', 'PO60_90', 'PO60_95']
order_2 = ['PO60_70', 'PO70_75', 'PO70_80', 'PO70_85', 'PO70_90', 'PO70_95']
df1 = df[df['Hemisphere']=='rh']
sns.boxplot(data=df1, x='Stimulus', y='EOG area (30ms)', width=0.8, fill=False, gap=.1, linewidth=2,
            saturation=0, color=color, order=order_2, ax=ax)
color='#9467bd' # #ff7f0e
sns.stripplot(data=df1, x='Stimulus', y='EOG area (30ms)',
            dodge=False, size=3, color=color, order=order_2, ax=ax)
ax.set_ylim([0, 1100])
ax.spines['top'].set_visible(False); ax.spines['right'].set_visible(False)

# plotting Figure 1 b
order_1 = ['PO60_70', 'PO60_75', 'PO60_80', 'PO60_85', 'PO60_90', 'PO60_95']
order_2 = ['PO60_70', 'PO70_75', 'PO70_80', 'PO70_85', 'PO70_90', 'PO70_95']
mask = df['Stimulus'].isin(order_1)
df1 = df[mask]
fig, ax = plt.subplots(1, 1, figsize=(10,4))
palette_color = ['#1f77b4', '#d62728'] 
sns.boxplot(data=df1, x='Stimulus', y='peak value (30ms)', hue='Hemisphere', width=0.8, fill=False, gap=.1, linewidth=2,
            saturation=0.75, palette=palette_color, order=order_1, ax=ax)
sns.stripplot(data=df1, x='Stimulus', y='peak value (30ms)', hue='Hemisphere',
            dodge=True, size=3, palette=palette_color, order=order_1, ax=ax, legend=False)
ax.spines['top'].set_visible(False); ax.spines['right'].set_visible(False)
ax.legend(frameon=False, loc='upper left')

# plotting Figure 1 c
fig, ax = plt.subplots(figsize=(11, 4))
colors = sns.color_palette('Set1')[1:5]
# colors = np.array(sns.color_palette('Set1'))[[1, 2, 6, 7, 3, 0, 8],:]
data = [19, 1, 2, 1]
# data = [11, 4, 2, 2, 2, 1, 1]
ingredients = ['transverstemporal-rh', 'bankssts-rh', 'entorhinal-rh', 'temporalpole-rh']
# ingredients = ['transverstemporal-lh', 'bankssts-lh', 'rostralanteriorcingulate-lh', 'paracentral-lh',
#                'entorhinal-lh', 'posteriorcingulate-lh', 'lateralorbitofrontal-lh']
def func(pct, allvals):
    absolute = int(np.round(pct/100.*np.sum(allvals)))
    return f"{pct:.1f}%"
wedges, autotexts = ax.pie(data, textprops=dict(color="w"), colors=colors)
legend_names = [f'{name} ({round(number/23*100, 1)}%)' for name, number in zip(ingredients, data)]
ax.legend(wedges, legend_names, loc="center left",
            bbox_to_anchor=(1, 0, 0.5, 1), frameon=False, fontsize=12)

# figure 1 d (needs to run previous parts)
grand_ev_dict['PO60_90'].plot_topomap(times=0.1, time_unit='ms', contours=6)

# figure 1 e
directory = '/Users/payamsadeghishabestari/KI_MEG/stcs/tinmeg1_morphed'
stc_files_list_rh = {'PO60_70': [], 'PO60_75': [], 'PO60_80': [],
                'PO60_85': [], 'PO60_90': [], 'PO60_95': []}
stc_files_list_lh = {'PO60_70': [], 'PO60_75': [], 'PO60_80': [],
                'PO60_85': [], 'PO60_90': [], 'PO60_95': []}
events = ['PO60_70', 'PO60_75', 'PO60_80', 'PO60_85', 'PO60_90', 'PO60_95']

brain_labels = mne.read_labels_from_annot(subject='fsaverage', parc='aparc')[:-1] # 68 labels
fs_dir = fetch_fsaverage(verbose=False)
src_fname = op.join(fs_dir, "bem", "fsaverage-ico-5-src.fif")
src = mne.read_source_spaces(src_fname, verbose=False)

for event in events:
    for filename in sorted(os.listdir(directory)): 
        f = os.path.join(directory, filename)
        if os.path.isfile(f) and f.endswith("-lh.stc") and event in f: # or -rh
                stc = mne.read_source_estimate(fname=f, subject='fsaverage')
                rh_data = stc.extract_label_time_course(brain_labels[-1], src, mode='mean', verbose=False)
                lh_data = stc.extract_label_time_course(brain_labels[-2], src, mode='mean', verbose=False)
                stc_files_list_rh[event].append(rh_data)
                stc_files_list_lh[event].append(lh_data)

fig, ax = plt.subplots(1, 1, figsize=(7,3))
lw = 0.5
colors = ['#1f77b4', '#d62728']
for event in events[:-1]:
    data = np.squeeze(np.array(stc_files_list_lh[event])).mean(axis=0)
    ax.plot(np.linspace(-300, 300, 151), data.T, label=event, linewidth=lw, color=colors[0])
    lw += 0.4
for event in events[-1:]:
    data = np.squeeze(np.array(stc_files_list_lh[event])).mean(axis=0)
    ax.plot(np.linspace(-300, 300, 151), data.T, label=event, linewidth=lw, color='k')
    lw += 0.4

ax.axvspan(50, 150, alpha=0.5, color='lightgrey')
ax.vlines(0, 0.5, 6, colors='black',linestyles='--')
for i in [-200, -100, 100, 200, 300]:
    ax.vlines(i, 0.5, 6, colors='black',linestyles=':', linewidth=0.5)

ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.legend(loc='upper right', ncols=6, fontsize=9, frameon=False, bbox_to_anchor=(0.6, 0.6, 0.6, 0.6))
ax.set_xlim([-310, 310])
ax.set_ylim([0, 6])

# extra plot
Brain = mne.viz.get_brain_class()
clr = 0.85
brain_kwargs = dict(alpha=1, background="white", cortex=[(clr, clr, clr), (clr, clr, clr)], size=(800, 600), views='lateral')
brain_labels = mne.read_labels_from_annot(subject='fsaverage', parc='aparc')[:-1] # 69 labels
brain = Brain("fsaverage", hemi="lh", surf="pial_semi_inflated", **brain_kwargs)
brain.add_label(brain_labels[-2], hemi="lh", color="#d62728", borders=False, alpha=0.9)
brain.show_view(roll=20, azimuth=30, elevation=80, distance=400)

#### figure 1 f
fig, axs = plt.subplots(1, 1, figsize=(6, 3))
time_array = np.linspace(-300, 300, 151)
stims = ['PO60_70', 'PO60_75', 'PO60_80', 'PO60_85', 'PO60_90', 'PO60_95']
color = '#ff7f0e'
lw = 0.5
for stim in stims[:-1]:
    axs.plot(time_array, abs(grand_ev_dict[stim].get_data(picks='EOG002')[0] * 1e6),
            linewidth=lw, label=stim, color=color)
    lw += 0.4
for stim in stims[-1:]:
    axs.plot(time_array, abs(grand_ev_dict[stim].get_data(picks='EOG002')[0] * 1e6),
            linewidth=lw, label=stim, color='k')
    
axs.axvspan(50, 180, alpha=0.4, color='lightgrey')
axs.legend(fontsize=9, frameon=False, bbox_to_anchor=(0.5, 0.1, 0.6, 0.6))
axs.spines['top'].set_visible(False); axs.spines['right'].set_visible(False)
for i in [-200, -100, 100, 200, 300]:
    axs.vlines(i, -10, 60, colors='black',linestyles=':', linewidth=0.5)
axs.vlines(0, -10, 60, colors='black',linestyles='--')
axs.set_ylabel(f'EOG amplitude at 70 dB (µv)')
axs.set_xlabel(f'Time (ms)')
axs.set_ylim([-10, 60])

In [ ]:
## figure 4 a (extra)
directory = '/Users/payamsadeghishabestari/KI_MEG/stcs/tinmeg1_morphed'
stc_files_list_rh = {'PO60_90': [], 'GP60_i240': []}
stc_files_list_lh = {'PO60_90': [], 'GP60_i240': []}
events = ['PO60_90', 'GP60_i240']

brain_labels = mne.read_labels_from_annot(subject='fsaverage', parc='aparc')[:-1] # 68 labels
fs_dir = fetch_fsaverage(verbose=False)
src_fname = op.join(fs_dir, "bem", "fsaverage-ico-5-src.fif")
src = mne.read_source_spaces(src_fname, verbose=False)

for event in events:
    for filename in sorted(os.listdir(directory)): 
        f = os.path.join(directory, filename)
        if os.path.isfile(f) and f.endswith("-lh.stc") and event in f: # or -rh
                stc = mne.read_source_estimate(fname=f, subject='fsaverage')
                rh_data = stc.extract_label_time_course(brain_labels[-1], src, mode='mean', verbose=False)
                lh_data = stc.extract_label_time_course(brain_labels[-2], src, mode='mean', verbose=False)
                stc_files_list_rh[event].append(rh_data)
                stc_files_list_lh[event].append(lh_data)

fig, ax = plt.subplots(1, 1, figsize=(7,3))
lw = None
# colors = ['#d62728', '#1f77b4']
colors = ['#1f77b4', '#d62728']

data = np.squeeze(np.array(stc_files_list_lh[events[0]])).mean(axis=0)
data_std = stats.sem(a=np.squeeze(np.array(stc_files_list_lh[event])), axis=0)
ax.plot(np.linspace(-300, 300, 151), data.T, label=events[0], linewidth=lw, color=colors[0])
ax.fill_between(np.linspace(-300, 300, 151), data.T - data_std.T,
                    data.T + data_std.T, color=colors[0], alpha=0.2, edgecolor="none")

data = np.squeeze(np.array(stc_files_list_lh[events[1]])).mean(axis=0)
data_std = stats.sem(a=np.squeeze(np.array(stc_files_list_lh[event])), axis=0)
ax.plot(np.linspace(-300, 300, 151), data.T, label=events[1], linewidth=lw, color=colors[0], linestyle='--')
ax.fill_between(np.linspace(-300, 300, 151), data.T - data_std.T,
                    data.T + data_std.T, color=colors[0], alpha=0.2, edgecolor="none")

data = np.squeeze(np.array(stc_files_list_rh[events[0]])).mean(axis=0)
data_std = stats.sem(a=np.squeeze(np.array(stc_files_list_rh[event])), axis=0)
ax.plot(np.linspace(-300, 300, 151), data.T, label=events[0], linewidth=lw, color=colors[1])
ax.fill_between(np.linspace(-300, 300, 151), data.T - data_std.T,
                    data.T + data_std.T, color=colors[1], alpha=0.2, edgecolor="none")

data = np.squeeze(np.array(stc_files_list_rh[events[1]])).mean(axis=0)
data_std = stats.sem(a=np.squeeze(np.array(stc_files_list_rh[event])), axis=0)
ax.plot(np.linspace(-300, 300, 151), data.T, label=events[1], linewidth=lw, color=colors[1], linestyle='--')
ax.fill_between(np.linspace(-300, 300, 151), data.T - data_std.T,
                    data.T + data_std.T, color=colors[1], alpha=0.2, edgecolor="none")


ax.axvspan(50, 150, alpha=0.5, color='lightgrey')
ax.vlines(0, 0.5, 6, colors='black',linestyles='--')
for i in [-200, -100, 100, 200, 300]:
    ax.vlines(i, 0.5, 6, colors='black',linestyles=':', linewidth=0.5)

ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.legend(loc='upper right', ncols=6, fontsize=9, frameon=False, bbox_to_anchor=(0.6, 0.6, 0.6, 0.6))
ax.set_xlim([-310, 310])
ax.set_ylim([0, 6])

In [ ]:
fig, axs = plt.subplots(1, 1, figsize=(6, 3))
time_array = np.linspace(-300, 300, 151)
stims = ['PO70_90', 'GP70_i240']
color = '#ff7f0e'
lw = None

eog_evs_po = []
for ev in np.array(evs)[:,4]: #PO70_90
    eog_evs_po.append(abs(ev.get_data(picks='EOG002')[0] * 1e6))

eog_evs_gp = []
for ev in np.array(evs)[:,14]: #GP70_i240
    eog_evs_gp.append(abs(ev.get_data(picks='EOG002')[0] * 1e6))

data = abs(np.array(eog_evs_po)).mean(axis=0)
data_std = stats.sem(a=abs(np.array(eog_evs_gp)), axis=0)
axs.plot(time_array, data, linewidth=lw, label=stim, color=color)
axs.fill_between(np.linspace(-300, 300, 151), data.T - data_std.T,
                    data.T + data_std.T, color=color, alpha=0.2, edgecolor="none")

data = abs(np.array(eog_evs_gp)).mean(axis=0)
data_std = stats.sem(a=abs(np.array(eog_evs_gp)), axis=0)
axs.plot(time_array, data, linewidth=lw, label=stim, color=color, linestyle='--')
axs.fill_between(np.linspace(-300, 300, 151), data.T - data_std.T,
                    data.T + data_std.T, color=color, alpha=0.2, edgecolor="none")
    
axs.axvspan(50, 180, alpha=0.4, color='lightgrey')
axs.legend(fontsize=9, frameon=False, bbox_to_anchor=(1.5, 0.1, 0.6, 0.6))
axs.spines['top'].set_visible(False); axs.spines['right'].set_visible(False)
for i in [-200, -100, 100, 200, 300]:
    axs.vlines(i, -10, 60, colors='black',linestyles=':', linewidth=0.5)
axs.vlines(0, -10, 60, colors='black',linestyles='--')
axs.set_ylabel(f'EOG amplitude at 70 dB (µv)')
axs.set_xlabel(f'Time (ms)')
axs.set_ylim([-10, 60])

Figure 2

In [ ]:
# plotting Figure 2 a
data1 = grand_ev_dict['GO_60'].get_data(picks='EOG002')[0] * 1e6
data2 = grand_ev_dict['GO_70'].get_data(picks='EOG002')[0] * 1e6

fig, ax = plt.subplots(1, 1, figsize=(6,3))
ax.plot(np.linspace(-300, 300, 151), abs(data1), label='GO_60', color='#ff7f0e', linewidth=2)
ax.plot(np.linspace(-300, 300, 151), abs(data2), label='GO_70', color='#9467bd', linewidth=2)
ax.vlines(0, -10, 60, colors='black',linestyles='--')
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.legend(loc='upper left', fontsize=9, frameon=False)
ax.set_ylim([-12, 62])
ax.set_xlim([-310, 310])
for i in [-200, -100, 100, 200, 300]:
    ax.vlines(i, -10, 60, colors='black',linestyles=':', linewidth=0.5)

# plotting Figure 2 b
directory = '/Users/payamsadeghishabestari/KI_MEG/stcs/tinmeg1_morphed'
stc_files_list_rh = {'GO_60': [], 'GO_70': []}
stc_files_list_lh = {'GO_60': [], 'GO_70': []}
events = ['GO_60', 'GO_70']

brain_labels = mne.read_labels_from_annot(subject='fsaverage', parc='aparc')[:-1] # 68 labels
fs_dir = fetch_fsaverage(verbose=False)
src_fname = op.join(fs_dir, "bem", "fsaverage-ico-5-src.fif")
src = mne.read_source_spaces(src_fname, verbose=False)

for event in events:
    for filename in sorted(os.listdir(directory)): 
        f = os.path.join(directory, filename)
        if os.path.isfile(f) and f.endswith("-lh.stc") and event in f and '697' not in f and '750' not in f and '853' not in f:
                stc = mne.read_source_estimate(fname=f, subject='fsaverage')
                rh_data = stc.extract_label_time_course(brain_labels[-1], src, mode='mean', verbose=False)
                lh_data = stc.extract_label_time_course(brain_labels[-2], src, mode='mean', verbose=False)
                stc_files_list_rh[event].append(rh_data)
                stc_files_list_lh[event].append(lh_data)

fig, ax = plt.subplots(1, 1, figsize=(6,3))
colors = ['#d62728', '#1f77b4'] 
for event, clr in zip(events, colors):
    data = np.squeeze(np.array(stc_files_list_rh[event])).mean(axis=0)
    # data_std = np.squeeze(np.array(stc_files_list_rh[event])).std(axis=0)
    data_std = stats.sem(a=np.squeeze(np.array(stc_files_list_rh[event])), axis=0)
    ax.plot(np.linspace(-300, 300, 151), data.T, label=event, color=clr)
    ax.fill_between(np.linspace(-300, 300, 151), data.T - data_std.T,
                    data.T + data_std.T, color=clr, alpha=0.1, edgecolor="none")
ax.vlines(0, 0.5, 8, colors='black',linestyles='--')
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.legend(loc='upper left', fontsize=9, frameon=False)
ax.set_xlim([-310, 310])
for i in [-200, -100, 100, 200, 300]:
    ax.vlines(i, 0.5, 8, colors='black',linestyles=':', linewidth=0.5)

# plotting Figure 2 c
fig, ax = plt.subplots(1, 1, figsize=(4,4))
# palette_color = 'Set1'
palette_color = ['#1f77b4', '#d62728'] 
order_1 = ['GO_60', 'GO_70']
order_2 = ['GO_70']
df1 = df[df['Hemisphere']=='rh']
sns.boxplot(data=df, x='Stimulus', y='peak value (30ms)', fill=False, linewidth=2, hue='Hemisphere', legend=False,
            saturation=0.6, palette=palette_color, order=order_1, gap=0.2, ax=ax)
sns.stripplot(data=df, x='Stimulus', y='peak value (30ms)', hue='Hemisphere', legend=False,
            dodge=True, size=4, palette=palette_color, order=order_1, ax=ax)
ax.set_ylim([0, 13])
ax.spines['top'].set_visible(False); ax.spines['right'].set_visible(False)

# plotting Figure 2 e
grand_ev_dict['GO_60'].plot_topomap(times=0.1, time_unit='ms', contours=6, vlim=(-240, 240))

# plotting Figure 2 d
brain_labels = mne.read_labels_from_annot(subject='fsaverage', parc='aparc')[:-1] # 68 labels
fig, ax = plt.subplots(1,1, figsize=(10, 4))
colors = np.array(sns.color_palette('Set1'))[[1, 2, 3, 4, 6, 0, 8],:]

# data = [17, 1, 1, 1, 2, 1]
data = [13, 2, 3, 1, 2, 1, 1]

# ingredients = ['transverstemporal-rh', 'bankssts-rh', 'entorhinal-rh', 'temporalpole-rh']
ingredients = [brain_labels[67].name, brain_labels[1].name,
                brain_labels[9].name, brain_labels[0].name, brain_labels[66].name,
                brain_labels[32].name, brain_labels[42].name]
def func(pct, allvals):
    absolute = int(np.round(pct/100.*np.sum(allvals)))
    return f"{pct:.1f}%"
wedges, autotexts = ax.pie(data, textprops=dict(color="w"), colors=colors)
legend_names = [f'{name} ({round(number/23*100, 1)}%)' for name, number in zip(ingredients, data)]
ax.legend(wedges, legend_names, loc="center left",
        bbox_to_anchor=(1, 0, 0.5, 1), frameon=False, fontsize=12)


Figure 3

In [ ]:
# plotting Figure 3 a
order_1 = ['GP70_i0', 'GP70_i60', 'GP70_i120', 'GP70_i240']
order_2 = ['GP60_i0', 'GP60_i60', 'GP60_i120', 'GP60_i240']
mask = df['Stimulus'].isin(order_1)
df1 = df[mask]
df2 = df1[df1['Hemisphere']=='rh']
fig, ax = plt.subplots(1, 1, figsize=(5,3))
palette_color = ['grey']
sns.boxplot(data=df2, x='Stimulus', y='EOG area inhibition', width=0.5, fill=False, linewidth=2,
            saturation=0.75, palette=palette_color, order=order_1, ax=ax)
sns.stripplot(data=df2, x='Stimulus', y='EOG area inhibition',
            dodge=False, size=3, palette=palette_color, order=order_1, ax=ax, legend=False)
ax.spines['top'].set_visible(False); ax.spines['right'].set_visible(False)
ax.legend(frameon=False, loc='upper right', bbox_to_anchor=(1.1, 1))
ax.set_ylim([-50, 100])
ax.hlines(0, -0.6, 3.6, colors='black',linestyles='--')
ax.hlines(50, -0.6, 3.6, colors='grey',linestyles=':')
ax.set_yticks([-50, 0, 50, 100])

# plotting Figure 3 b
order_1 = ['GP60_i0', 'GP60_i60', 'GP60_i120', 'GP60_i240']
order_2 = ['GP60_i0', 'GP60_i60', 'GP60_i120', 'GP60_i240']
mask = df['Stimulus'].isin(order_2)
df1 = df[mask]
fig, ax = plt.subplots(1, 1, figsize=(9,3))
palette_color = ['#1f77b4', '#d62728']
sns.boxplot(data=df1, x='Stimulus', y='area inhibition (30ms)', hue='Hemisphere', width=0.8, fill=False, gap=.1, linewidth=2,
            saturation=0.75, palette=palette_color, order=order_2, ax=ax)
sns.stripplot(data=df1, x='Stimulus', y='area inhibition (30ms)', hue='Hemisphere',
            dodge=True, size=3, palette=palette_color, order=order_2, ax=ax, legend=False)
ax.spines['top'].set_visible(False); ax.spines['right'].set_visible(False)
ax.legend(frameon=False, loc='upper right', bbox_to_anchor=(1.1, 1))
ax.set_ylim([-50, 100])
ax.hlines(0, -0.6, 3.6, colors='black',linestyles='--')
ax.hlines(50, -0.6, 3.6, colors='grey',linestyles=':')
ax.set_yticks([-50, 0, 50, 100])


Figure 4

In [ ]:
# load dataframe and remove three subjects
df = pd.read_csv('/Users/payamsadeghishabestari/KI_MEG/dataframes/tinmeg2_transverstemporal.csv') 

# plotting Figure 4 a 
order_1 = ['PO_00', 'PO_03', 'PO_08', 'PO_30', 'PO_33', 'PO_38', 'PO_80', 'PO_83', 'PO_88']

mask = df['Stimulus'].isin(order_1)
df1 = df[mask]
fig, ax = plt.subplots(1, 1, figsize=(7,3))
palette_color = ['#d62728', '#d62728', '#d62728',
                '#1f77b4', '#1f77b4', '#1f77b4',
                '#2ca02c', '#2ca02c', '#2ca02c']
sns.boxplot(data=df1, x='Stimulus', y='EOG peak', width=0.3, fill=False, linewidth=2,
            saturation=0.55, palette=palette_color, order=order_1, ax=ax)
sns.stripplot(data=df1, x='Stimulus', y='EOG peak',
            dodge=False, size=3, palette=palette_color, order=order_1, ax=ax, legend=False)
ax.spines['top'].set_visible(False); ax.spines['right'].set_visible(False)
ax.legend(frameon=False, loc='upper right', bbox_to_anchor=(1.1, 1))

# plotting Figure 4 b 
order_1 = ['PO_00', 'PO_03', 'PO_08', 'PO_30', 'PO_33', 'PO_38', 'PO_80', 'PO_83', 'PO_88']
mask = df['Stimulus'].isin(order_1)
df1 = df[mask]
fig, ax = plt.subplots(1, 1, figsize=(11,3))
palette_color = ['#d62728', '#1f77b4']
sns.boxplot(data=df1, x='Stimulus', y='peak value (30ms)', hue='Hemisphere',
            width=0.5, fill=False, linewidth=2, saturation=0, palette=palette_color, gap=.1,
            order=order_1, ax=ax)
sns.stripplot(data=df1, x='Stimulus', y='peak value (30ms)', hue='Hemisphere',
            dodge=True, size=3, palette=palette_color, order=order_1, ax=ax, legend=False)
ax.spines['top'].set_visible(False); ax.spines['right'].set_visible(False)
ax.legend(frameon=False, loc='upper right', bbox_to_anchor=(1.1, 1))

# plotting Figure 4 c
order_2 = ['GO_00', 'GO_03', 'GO_08', 'GO_30', 'GO_33', 'GO_38', 'GO_80', 'GO_83', 'GO_88']
mask = df['Stimulus'].isin(order_2)
df1 = df[mask]
fig, ax = plt.subplots(1, 1, figsize=(11,3))
palette_color = ['#d62728', '#1f77b4']
sns.boxplot(data=df1, x='Stimulus', y='peak value (30ms)', hue='Hemisphere',
            width=0.5, fill=False, linewidth=2, saturation=0, palette=palette_color, gap=.1,
            order=order_2, ax=ax)
sns.stripplot(data=df1, x='Stimulus', y='peak value (30ms)', hue='Hemisphere',
            dodge=True, size=3, palette=palette_color, order=order_2, ax=ax, legend=False)
ax.spines['top'].set_visible(False); ax.spines['right'].set_visible(False)
ax.legend(frameon=False, loc='upper right', bbox_to_anchor=(1.1, 1))
ax.set_ylim([0, 8.8])

Tables

In [ ]:
# table 1
evs1 = ['PO60_70', 'PO60_75', 'PO60_80', 'PO60_85', 'PO60_90', 'PO60_95']
evs2 = ['PO70_75', 'PO70_80', 'PO70_85', 'PO70_90', 'PO70_95']
for ev in evs2:
    df1 = df[df['Stimulus']==ev]
    df2 = df1[df1['Hemisphere']=='lh']
    mean = df2['EOG peak latency'].mean()
    std = df2['EOG peak latency'].std()
    print(f'{round(mean, 2)} + {round(std, 2)}')

In [ ]:
# table 2
df1 = df[df['Stimulus']=='GO_60']
df2 = df1[df1['Hemisphere']=='rh']
median = round(df2['latency'].median(), 2)
std = round(df2["latency"].std(), 2)
print(f'{median} + {std}')

In [ ]:
# table 3
evs1 = ['GP60_i0', 'GP60_i60', 'GP60_i120', 'GP60_i240']
evs2 = ['GP70_i0', 'GP70_i60', 'GP70_i120', 'GP70_i240']
for ev in evs2:
    df1 = df[df['Stimulus']==ev]
    df2 = df1[df1['Hemisphere']=='lh']
    mean = df2['amplitude inhibition (30ms)'].median()
    std = df2['amplitude inhibition (30ms)'].std()
    print(f'{round(mean, 2)} + {round(std, 2)}')

Tinmeg3 plots

In [ ]:
events = ['GPP_00', 'GPG_00', 'PO_00', 'GO_00', 'PPP_00', 'PPG_00',
        'GPP_03', 'GPG_03', 'PO_03', 'GO_03',
        'GPP_08', 'GPG_08', 'PO_08', 'GO_08']
# load dataframe and remove three subjects
df = pd.read_csv('/Users/payamsadeghishabestari/KI_MEG/dataframes/tinmeg3_transverstemporal.csv') 

In [ ]:
# plotting 1
order_1 = ['GPP_00', 'GPP_03', 'GPP_08']
order_2 = ['GPG_00', 'GPG_03', 'GPG_08']

mask = df['Stimulus'].isin(order_2)
df1 = df[mask]
fig, ax = plt.subplots(1, 1, figsize=(6,4))
palette_color = ['#d62728', '#1f77b4']
sns.boxplot(data=df1, x='Stimulus', y='area inhibition (30ms)', hue='Hemisphere', width=0.8, fill=False, gap=.1, linewidth=2,
            saturation=0.75, palette=palette_color, order=order_2, ax=ax)
sns.stripplot(data=df1, x='Stimulus', y='area inhibition (30ms)', hue='Hemisphere',
            dodge=True, size=3, palette=palette_color, order=order_2, ax=ax, legend=False)
ax.spines['top'].set_visible(False); ax.spines['right'].set_visible(False)
ax.legend(frameon=False, loc='upper right', bbox_to_anchor=(1.1, 1))
ax.set_ylim([-100, 100])
ax.hlines(0, -0.6, 2.6, colors='black',linestyles='--')
ax.hlines(50, -0.6, 2.6, colors='grey',linestyles=':')

# plotting 2
directory = '/Users/payamsadeghishabestari/KI_MEG/stcs/tinmeg3_morphed'
ev_name = 'GPP'
stc_files_list_rh = {f'{ev_name}_00': [], f'{ev_name}_03': [], f'{ev_name}_08': []}
stc_files_list_lh = {f'{ev_name}_00': [], f'{ev_name}_03': [], f'{ev_name}_08': []}
events = [f'{ev_name}_00', f'{ev_name}_03', f'{ev_name}_08']

brain_labels = mne.read_labels_from_annot(subject='fsaverage', parc='aparc')[:-1] # 68 labels
fs_dir = fetch_fsaverage(verbose=False)
src_fname = op.join(fs_dir, "bem", "fsaverage-ico-5-src.fif")
src = mne.read_source_spaces(src_fname, verbose=False)

for event in events:
    for filename in sorted(os.listdir(directory)): 
        f = os.path.join(directory, filename)
        if os.path.isfile(f) and f.endswith("-lh.stc") and event in f: # or -rh
                stc = mne.read_source_estimate(fname=f, subject='fsaverage')
                rh_data = stc.extract_label_time_course(brain_labels[-1], src, mode='mean', verbose=False)
                lh_data = stc.extract_label_time_course(brain_labels[-2], src, mode='mean', verbose=False)
                stc_files_list_rh[event].append(rh_data)
                stc_files_list_lh[event].append(lh_data)

fig, ax = plt.subplots(1, 1, figsize=(6,3))
colors = ['#d62728', '#1f77b4', '#2ca02c'] 
for event, clr in zip(events, colors):
    data = np.squeeze(np.array(stc_files_list_lh[event])).mean(axis=0)
    ax.plot(np.linspace(-300, 300, 151), data.T, label=event, color=clr)
ax.vlines(0, 0.5, 6, colors='black',linestyles='--')
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.legend(loc='upper right', fontsize=9, frameon=False)
ax.set_xlim([-310, 310])
for i in [-300, -200, -100, 100, 200, 300]:
    ax.vlines(i, 0.5, 6, colors='black',linestyles=':', linewidth=0.5)

Statistics on the dataframe

In [ ]:
# choose what two columns to compare (EOG inhibition)
df1 = df[df['Hemisphere']=='rh']
for ev in ['GP70_i0', 'GP70_i60', 'GP70_i120', 'GP70_i240']:
    column1 = df1[df1['Stimulus']=='PO70_90']['EOG area inhibition'] 
    column2 = df1[df1['Stimulus']==ev]['EOG area inhibition']
    # Perform the t-test and df
    t_stat, p_value = stats.ttest_ind(column1, column2, permutations=10000)
    degree_freedom = len(column1) + len(column2) - 2
    print(f'event={ev}, (df)={degree_freedom}, t_value={t_stat}, p_value={round(p_value,5)}')

New series of plots

In [ ]:
# Create epochs dictionary (some needs concatenating)
epochs_folder = '/Users/payamsadeghishabestari/KI_MEG/epochs/tinmeg2/epochs_bads_dropped'
epochs_file = {}
for f in sorted(os.listdir(epochs_folder)):
    file = os.path.join(epochs_folder, f)
    if file.endswith("-epo.fif") and '697' not in file and '750' not in file and '853' not in file:
        epochs_file[f'{file[-11:-8]}'] = file

# compute evoked objects, and making grand average dictionary
evs = []
for ep_f in tqdm(list(epochs_file.values())):
    evs.append(mne.read_epochs(fname=ep_f, verbose=False).average(picks=['meg', 'eog'], by_event_type=True))

grnd_ev_dict = {}
for stim_idx, stim in enumerate(list(events_dict_tinmeg2.keys())):
    evs_stim = []
    for ev in evs:
        evs_stim.append(ev[stim_idx])
    grnd_ev_dict[stim] = evs_stim

grand_ev_dict_tinmeg2 = {}
for stim in list(grnd_ev_dict.keys()):
    grand_ev_dict_tinmeg2[stim] = mne.grand_average(grnd_ev_dict[stim])

# Create epochs dictionary (some needs concatenating)
epochs_folder = '/Users/payamsadeghishabestari/KI_MEG/epochs/tinmeg3/epochs_bads_dropped'
epochs_file = {}
for f in sorted(os.listdir(epochs_folder)):
    file = os.path.join(epochs_folder, f)
    if file.endswith("-epo.fif") and '697' not in file and '750' not in file and '853' not in file:
        epochs_file[f'{file[-12:-8]}'] = file

# compute evoked objects, and making grand average dictionary
evs = []
for ep_f in tqdm(list(epochs_file.values())):
    evs.append(mne.read_epochs(fname=ep_f, verbose=False).average(picks=['meg', 'eog'], by_event_type=True))

grnd_ev_dict = {}
for stim_idx, stim in enumerate(list(events_dict_tinmeg3.keys())):
    evs_stim = []
    for ev in evs:
        evs_stim.append(ev[stim_idx])
    grnd_ev_dict[stim] = evs_stim

grand_ev_dict_tinmeg3 = {}
for stim in list(grnd_ev_dict.keys()):
    grand_ev_dict_tinmeg3[stim] = mne.grand_average(grnd_ev_dict[stim])

In [ ]:
# plotting plots 1, 2, 3 from firsl line
fig, ax = plt.subplots(1, 1, figsize=(6,3))
colors = ['#ff7f0e', '#9467bd'] 
data1 = grand_ev_dict_tinmeg2['PO_08'].get_data(picks='EOG002')[0]
data2 = grand_ev_dict_tinmeg2['GPP_08'].get_data(picks='EOG002')[0]
data3 = grand_ev_dict_tinmeg3['PO_08'].get_data(picks='EOG002')[0]
data4 = grand_ev_dict_tinmeg3['GPP_08'].get_data(picks='EOG002')[0]


ax.plot(np.linspace(-300, 300, 151), data1*1e6, color=colors[0], label='Control (PO)')
ax.plot(np.linspace(-300, 300, 151), data2*1e6, color=colors[1], label='Control (GPP)')
ax.plot(np.linspace(-300, 300, 151), data3*1e6, linestyle='--', color=colors[0], label='Tinnitus (PO)')
ax.plot(np.linspace(-300, 300, 151), data4*1e6, linestyle='--', color=colors[1], label='Tinnitus (GPP)')

ax.vlines(0, -10, 30, colors='black',linestyles='--')
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.legend(loc='best', fontsize=9, frameon=False)
ax.set_xlim([-310, 310])
ax.set_ylim([-10, 30])
# for i in [-200, -100, 100, 200, 300]:
#     ax.vlines(i, -10, 30, colors='black',linestyles=':', linewidth=0.5)

In [ ]:
# plotting plots 1, 2, 3 from second and third line (you just need to set event name in ax function)
# plotting 2
directory1 = '/Users/payamsadeghishabestari/KI_MEG/stcs/tinmeg2_morphed'
directory2 = '/Users/payamsadeghishabestari/KI_MEG/stcs/tinmeg3_morphed'
events = ['PO_00', 'PO_03', 'PO_08', 'GPP_00', 'GPP_03', 'GPP_08']

brain_labels = mne.read_labels_from_annot(subject='fsaverage', parc='aparc')[:-1] # 68 labels
fs_dir = fetch_fsaverage(verbose=False)
src_fname = op.join(fs_dir, "bem", "fsaverage-ico-5-src.fif")
src = mne.read_source_spaces(src_fname, verbose=False)

tinmeg2_stc_files_list_lh = {'PO_00': [], 'PO_03': [], 'PO_08': [], 'GPP_00': [], 'GPP_03': [], 'GPP_08': []}
tinmeg2_stc_files_list_rh = {'PO_00': [], 'PO_03': [], 'PO_08': [], 'GPP_00': [], 'GPP_03': [], 'GPP_08': []}
tinmeg3_stc_files_list_lh = {'PO_00': [], 'PO_03': [], 'PO_08': [], 'GPP_00': [], 'GPP_03': [], 'GPP_08': []}
tinmeg3_stc_files_list_rh = {'PO_00': [], 'PO_03': [], 'PO_08': [], 'GPP_00': [], 'GPP_03': [], 'GPP_08': []}

for ev in events:
    for filename in sorted(os.listdir(directory1)): 
        f = os.path.join(directory1, filename)
        if os.path.isfile(f) and f.endswith("-lh.stc") and ev in f: # or -rh
            stc = mne.read_source_estimate(fname=f, subject='fsaverage')
            rh_data = stc.extract_label_time_course(brain_labels[-1], src, mode='mean', verbose=False)
            lh_data = stc.extract_label_time_course(brain_labels[-2], src, mode='mean', verbose=False)
            tinmeg2_stc_files_list_rh[ev].append(rh_data)
            tinmeg2_stc_files_list_lh[ev].append(lh_data)

for ev in events:
    for filename in sorted(os.listdir(directory2)): 
        f = os.path.join(directory2, filename)
        if os.path.isfile(f) and f.endswith("-lh.stc") and ev in f: # or -rh
            stc = mne.read_source_estimate(fname=f, subject='fsaverage')
            rh_data = stc.extract_label_time_course(brain_labels[-1], src, mode='mean', verbose=False)
            lh_data = stc.extract_label_time_course(brain_labels[-2], src, mode='mean', verbose=False)
            tinmeg3_stc_files_list_rh[ev].append(rh_data)
            tinmeg3_stc_files_list_lh[ev].append(lh_data)


fig, ax = plt.subplots(1, 1, figsize=(6,3))
colors = ['#1f77b4', '#d62728'] 
data1 = np.squeeze(np.array(tinmeg2_stc_files_list_rh['PO_00'])).mean(axis=0)
data2 = np.squeeze(np.array(tinmeg2_stc_files_list_rh['GPP_00'])).mean(axis=0)
data3 = np.squeeze(np.array(tinmeg3_stc_files_list_rh['PO_00'])).mean(axis=0)
data4 = np.squeeze(np.array(tinmeg3_stc_files_list_rh['GPP_00'])).mean(axis=0)

ax.plot(np.linspace(-300, 300, 151), data1.T, color=colors[0])
ax.plot(np.linspace(-300, 300, 151), data2.T, color=colors[1])
ax.plot(np.linspace(-300, 300, 151), data3.T, color=colors[0], linestyle='--')
ax.plot(np.linspace(-300, 300, 151), data4.T, color=colors[1], linestyle='--')

ax.vlines(0, 0, 6, colors='black',linestyles='--')
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.set_xlim([-310, 310])
for i in [-200, -100, 100, 200, 300]:
    ax.vlines(i, 0.5, 6, colors='black',linestyles=':', linewidth=0.5)

In [ ]:
# plotting inhibitions in tinmeg2 and tinmeg3 for GPP
df_tinmeg2 = pd.read_csv('/Users/payamsadeghishabestari/KI_MEG/dataframes/tinmeg2_transverstemporal.csv')
df_tinmeg3 = pd.read_csv('/Users/payamsadeghishabestari/KI_MEG/dataframes/tinmeg3_transverstemporal.csv')
# updating dataframes and concatenating them
df_tinmeg2 = df_tinmeg2.drop('EOG peak latency', axis=1)
df_tinmeg2['Participants'] = len(df_tinmeg2) * ['tinmeg2']
df_tinmeg3['Participants'] = len(df_tinmeg3) * ['tinmeg3']
df = pd.concat([df_tinmeg2, df_tinmeg3])

In [ ]:
# plotting inhibitions in tinmeg2 and tinmeg3 for GPP (EOG)
order = ['GPP_00', 'GPP_03', 'GPP_08']
df1 = df[df['Hemisphere']=='rh']
mask = df1['Stimulus'].isin(order)
df2 = df1[mask]
palette_colors = ['#ff7f0e', '#9467bd'] 
fig, ax = plt.subplots(1, 1, figsize=(7,4))
# boxprops = {'linestyle': '-', 'linewidth': 2, 'color': 'grey'}
sns.boxplot(data=df2, x='Stimulus', y='EOG area inhibition', hue='Participants', width=0.8, fill=False, gap=.1, linewidth=2,
            saturation=0, palette=palette_colors, order=order, ax=ax)
sns.stripplot(data=df2, x='Stimulus', y='EOG area inhibition', hue='Participants',
            dodge=True, size=3, palette=palette_colors, order=order, ax=ax, legend=False)
ax.set_ylim([-150, 150])
ax.spines['top'].set_visible(False); ax.spines['right'].set_visible(False)
ax.legend(frameon=False, loc='upper right')
ax.hlines(0, -0.6, 2.6, colors='black',linestyles='--')

# plotting inhibitions in tinmeg2 and tinmeg3 for GPP (left hemisphere)
order = ['GPP_00', 'GPP_03', 'GPP_08']
df1 = df[df['Hemisphere']=='lh']
mask = df1['Stimulus'].isin(order)
df2 = df1[mask]
palette_colors = ['#1f77b4', '#d62728']
fig, ax = plt.subplots(1, 1, figsize=(7,4))
# boxprops = {'linestyle': '-', 'linewidth': 2, 'color': 'grey'}
sns.boxplot(data=df2, x='Stimulus', y='area inhibition (30ms)', hue='Participants', width=0.8, fill=False, gap=.1, linewidth=2,
            saturation=0, palette=palette_colors, order=order, ax=ax)
sns.stripplot(data=df2, x='Stimulus', y='area inhibition (30ms)', hue='Participants',
            dodge=True, size=3, palette=palette_colors, order=order, ax=ax, legend=False)
ax.set_ylim([-150, 150])
ax.spines['top'].set_visible(False); ax.spines['right'].set_visible(False)
ax.legend(frameon=False, loc='upper right')
ax.hlines(0, -0.6, 2.6, colors='black',linestyles='--')

# plotting inhibitions in tinmeg2 and tinmeg3 for GPP (right hemisphere)
order = ['GPP_00', 'GPP_03', 'GPP_08']
df1 = df[df['Hemisphere']=='rh']
mask = df1['Stimulus'].isin(order)
df2 = df1[mask]
palette_colors = ['#1f77b4', '#d62728']
fig, ax = plt.subplots(1, 1, figsize=(7,4))
# boxprops = {'linestyle': '-', 'linewidth': 2, 'color': 'grey'}
sns.boxplot(data=df2, x='Stimulus', y='area inhibition (30ms)', hue='Participants', width=0.8, fill=False, gap=.1, linewidth=2,
            saturation=0, palette=palette_colors, order=order, ax=ax)
sns.stripplot(data=df2, x='Stimulus', y='area inhibition (30ms)', hue='Participants',
            dodge=True, size=3, palette=palette_colors, order=order, ax=ax, legend=False)
ax.set_ylim([-150, 150])
ax.spines['top'].set_visible(False); ax.spines['right'].set_visible(False)
ax.legend(frameon=False, loc='upper right')
ax.hlines(0, -0.6, 2.6, colors='black',linestyles='--')

In [ ]:
# plotting gap only area values
df1 = df[df['Hemisphere']=='rh'] # choosing the hemisphere
df2 = df1[df1['Participants']=='tinmeg3'] # choosing only tinmeg3
df3 = df2[df2['Stimulus']=='GO_08'] # choosing the stimulus for tinmeg3
order = ['GO_08', 'GO_38', 'GO_88']
mask = df1['Stimulus'].isin(order) # choosing the events for tinmeg2
df4 = df1[mask]
df5 = df4[df4['Participants']=='tinmeg2']
df6 = pd.concat([df3, df5])

fig, ax = plt.subplots(1, 1, figsize=(7,4))
sns.barplot(data=df6, x='Stimulus', y='area value (30ms)', hue='Participants', fill=False, gap=.5, linewidth=2,
            capsize=.1, err_kws={"color": ".5", "linewidth": 2.5}, width=1.2, saturation=0, palette=palette_colors,
            order=order, legend=False, ax=ax)
ax.legend(frameon=False)
ax.spines['top'].set_visible(False); ax.spines['right'].set_visible(False)
ax.set_ylim([0, 70])

In [ ]:
# plotting inhibition values for different noise carrier conditions (NBN and BBN)
df1 = df[df['Hemisphere']=='rh'] # choosing the hemisphere
df2 = df1[df1['Participants']=='tinmeg2'] # choosing only tinmeg2
order = ['GPP_08', 'GPP_38', 'GPP_88']
mask = df2['Stimulus'].isin(order) # choosing the events for tinmeg2
df3 = df2[mask]

# palette_colors1 = ['#ff7f0e', '#9467bd', '#17becf']
palette_colors1 = ['#1f77b4', '#2ca02c', '#d62728']
palette_colors2 = ['k', 'k', 'k']
fig, ax = plt.subplots(1, 1, figsize=(4,4))
sns.boxplot(data=df3, x='Stimulus', y='area inhibition (30ms)', width=0.8, fill=True, gap=.1, linewidth=2,
            saturation=0.75, palette=palette_colors1, order=order, ax=ax)
sns.stripplot(data=df3, x='Stimulus', y='EOG area inhibition',
            dodge=False, size=3, palette=palette_colors2, order=order, ax=ax)
ax.legend(frameon=False)
ax.spines['top'].set_visible(False); ax.spines['right'].set_visible(False)
ax.set_ylim([-150, 120])
ax.hlines(0, -0.6, 2.6, colors='black',linestyles='--')

In [ ]:
# extracting table values to estimate ICC
df_tinmeg1 = pd.read_csv('/Users/payamsadeghishabestari/KI_MEG/dataframes/tinmeg1_transverstemporal.csv')
df_tinmeg2 = pd.read_csv('/Users/payamsadeghishabestari/KI_MEG/dataframes/tinmeg2_transverstemporal.csv')

# tinmeg1
df1 = df_tinmeg1[df_tinmeg1['Stimulus']=='GP60_i240']
df2 = df1[df1['Hemisphere']=='rh']
order = [539, 842, 849, 840, 863, 862, 850, 832, 861]
mask = df2['subject ID'].isin(order) # choosing the events for tinmeg2
df3 = df2[mask]
df3[['subject ID', 'area inhibition (30ms)']]

# tinmeg2
df1 = df_tinmeg2[df_tinmeg2['Stimulus']=='PO_00']
df2 = df1[df1['Hemisphere']=='lh']
order = [916, 979, 981, 982, 984, 983, 980, 986, 988]
mask = df2['subject ID'].isin(order) # choosing the events for tinmeg2
df3 = df2[mask]
df3[['subject ID', 'area value (30ms)']]

new series of plots after fixing tinmeg2

In [ ]:
sub_list = [916, 979, 980, 982, 983, 986, 988] # drop 981, 984
evs_gpp = []; evs_gpp_3 = []; evs_gpp_8 = []
evs_po = []; evs_po_3 = []; evs_po_8 = []
for id in sub_list:
    fname = f'/Users/payamsadeghishabestari/KI_MEG/epochs/tinmeg2/epochs_bads_dropped/epochs_subject_{id}-epo.fif'
    epochs = mne.read_epochs(fname=fname, preload=True)
    ev = epochs.average(picks=['meg', 'eog'], by_event_type=True)
    evs_gpp.append(ev[0]) 
    evs_po.append(ev[2])
    evs_gpp_3.append(ev[6]) 
    evs_po_3.append(ev[8])
    evs_gpp_8.append(ev[10])  
    evs_po_8.append(ev[12])

grand_ev_gpp = mne.grand_average(evs_gpp)
grand_ev_po = mne.grand_average(evs_po)
grand_ev_gpp_3 = mne.grand_average(evs_gpp_3)
grand_ev_po_3 = mne.grand_average(evs_po_3)
grand_ev_gpp_8 = mne.grand_average(evs_gpp_8)
grand_ev_po_8 = mne.grand_average(evs_po_8)

In [ ]:
# traces in max channel
fig, ax = plt.subplots(1, 1, figsize=(6,3))
data1 = grand_ev_po_8.get_data(picks='MEG1332')
data2 = grand_ev_gpp_8.get_data(picks='MEG1332')

ax.plot(np.linspace(-300, 300, 151), data1.T * 1e13, color=colors[0], label='PO')
ax.plot(np.linspace(-300, 300, 151), data2.T * 1e13, color=colors[1], label='GPP')
ax.vlines(0, -50, 120, colors='black',linestyles='--')
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.legend(loc='best', fontsize=9, frameon=False)
ax.set_xlim([-310, 310])
ax.set_ylim([-50, 120])
for i in [-200, -100, 100, 200, 300]:
    ax.vlines(i, -50, 120, colors='black',linestyles=':', linewidth=0.5)

Sensor plots of tinmeg1 for supplementary

In [ ]:
# Create epochs dictionary (some needs concatenating)
epochs_folder = '/Users/payamsadeghishabestari/KI_MEG/epochs/tinmeg1'
epochs_file = {}
for f in sorted(os.listdir(epochs_folder)):
    file = os.path.join(epochs_folder, f)
    if file.endswith("-epo.fif"):
        epochs_file[f'{file[-11:-8]}'] = file

# Create epochs dictionary (some needs concatenating)
epochs_folder = '/Users/payamsadeghishabestari/KI_MEG/epochs/tinmeg1'
epochs_file = {}
for f in sorted(os.listdir(epochs_folder)):
    file = os.path.join(epochs_folder, f)
    if file.endswith("-epo.fif") and '697' not in file and '750' not in file and '853' not in file:
        epochs_file[f'{file[-11:-8]}'] = file

# compute evoked objects, and making grand average dictionary
evs = []
for ep_f in tqdm(list(epochs_file.values())):
    evs.append(mne.read_epochs(fname=ep_f, verbose=False).average(picks=['meg', 'eog'], by_event_type=True))

grnd_ev_dict = {}
for stim_idx, stim in enumerate(list(events_dict_tinmeg1.keys())):
    evs_stim = []
    for ev in evs:
        evs_stim.append(ev[stim_idx])
    grnd_ev_dict[stim] = evs_stim

grand_ev_dict = {}
for stim in list(grnd_ev_dict.keys()):
    grand_ev_dict[stim] = mne.grand_average(grnd_ev_dict[stim])

# separate chanels on left and right
info_ch = evs[0][0].info['chs']
meg_chs_right = []; meg_chs_left = []
grad_chs_right = []; grad_chs_left = []
for i in range(len(info_ch)):
    if info_ch[i]['unit'] == 112: # meg code
        if info_ch[i]['loc'][0] > 0:
            meg_chs_right.append(info_ch[i]['ch_name'])
        if info_ch[i]['loc'][0] < 0:
            meg_chs_left.append(info_ch[i]['ch_name'])
    if info_ch[i]['unit'] == 201: # grad code
        if info_ch[i]['loc'][0] > 0:
            grad_chs_right.append(info_ch[i]['ch_name'])
        if info_ch[i]['loc'][0] < 0:
            grad_chs_left.append(info_ch[i]['ch_name'])

# select the left/ channels with largest ptp amplitude
ev_data_left = grand_ev_dict['PO60_70'].get_data(picks=grad_chs_left)
ev_data_right = grand_ev_dict['PO60_70'].get_data(picks=grad_chs_right)
max_values = []
for ch_idx in range(len(ev_data_left)):
    max_values.append(ev_data_left[ch_idx][50:150].max())
ch_max_left = grad_chs_left[np.argmax(np.array(max_values))]
max_values = []
for ch_idx in range(len(ev_data_right)):
    max_values.append(ev_data_right[ch_idx][50:150].max())
ch_max_right = grad_chs_right[np.argmax(np.array(max_values))]

Making a dataframe for it

In [ ]:
# making big dataframe for transverstemporal rh/lh
my_dict = {'subject ID': [], 'Stimulus': [], 'Ch_name': [],
        'peak value (30ms)': [], 'peak value (100ms)': [], 'area value (30ms)': [],
        'area value (100ms)': [], 'amplitude inhibition (30ms)': [],
        'area inhibition (30ms)': [], 'amplitude inhibition (100ms)': [],
        'area inhibition (100ms)': []} 

events = ['PO60_70', 'PO60_75', 'PO60_80', 'PO60_85', 'PO60_90', 'PO60_95',
        'PO70_75', 'PO70_80', 'PO70_85', 'PO70_90', 'PO70_95',
        'GP60_i0', 'GP60_i60', 'GP60_i120', 'GP60_i240',
        'GP70_i0', 'GP70_i60', 'GP70_i120', 'GP70_i240',
        'GO_60', 'GO_70']

subjects = ['539', '756', '832', '835', '836', '838', '839', '840',
        '841', '842', '844', '845', '847', '849','850', '852',
        '856', '857', '858', '859', '861', '862', '863'] 

chs = [ch_max_left, ch_max_right]
combinations = product(subjects, events, chs)

for subject, stim, ch in tqdm(combinations):
                
        # reading epoch file 
        fname = epochs_file[subject]
        ep = mne.read_epochs(fname=fname, preload=True, verbose=False)[stim]
        ep_data = np.squeeze(ep.get_data(picks=ch)).mean(axis=0) 

        # computing some params  
        ep_peak_100ms = ep_data[87:114].max()
        ep_peak_30ms = ep_data[96:105].max()
        ep_area_100ms = ep_data[87:114].sum()
        ep_area_30ms = ep_data[96:105].sum()
        
        if '60' in stim and '70' not in stim:
                ep_stn = mne.read_epochs(fname=fname, preload=True, verbose=False)['PO60_90']
                ep_stn_data = np.squeeze(ep_stn.get_data(picks=ch)).mean(axis=0)
                ep_stn_peak_100ms = ep_stn_data[87:114].max()
                ep_stn_peak_30ms = ep_stn_data[96:105].max()
                ep_stn_area_100ms = ep_stn_data[87:114].sum()
                ep_stn_area_30ms = ep_stn_data[96:105].sum()
        if '70' in stim:
                ep_stn = mne.read_epochs(fname=fname, preload=True, verbose=False)['PO70_90']
                ep_stn_data = np.squeeze(ep_stn.get_data(picks=ch)).mean(axis=0)
                ep_stn_peak_100ms = ep_stn_data[87:114].max()
                ep_stn_peak_30ms = ep_stn_data[96:105].max()
                ep_stn_area_100ms = ep_stn_data[87:114].sum()
                ep_stn_area_30ms = ep_stn_data[96:105].sum()

        # computing inhibition indexes 
        my_dict['amplitude inhibition (30ms)'].append((1 - (ep_peak_30ms / ep_stn_peak_30ms)) * 100)
        my_dict['area inhibition (30ms)'].append((1 - (ep_area_30ms / ep_stn_area_30ms)) * 100)
        my_dict['amplitude inhibition (100ms)'].append((1 - (ep_peak_100ms / ep_stn_peak_100ms)) * 100)
        my_dict['area inhibition (100ms)'].append((1 - (ep_area_100ms / ep_stn_area_100ms)) * 100)

        # putting in the dictionary
        my_dict['subject ID'].append(subject)
        my_dict['Stimulus'].append(stim)
        my_dict['Ch_name'].append(ch)
        my_dict['peak value (30ms)'].append(ep_peak_30ms)
        my_dict['peak value (100ms)'].append(ep_peak_100ms)
        my_dict['area value (30ms)'].append(ep_area_30ms)
        my_dict['area value (100ms)'].append(ep_area_100ms)

df = pd.DataFrame(my_dict)
# save it
df.to_csv('/Users/payamsadeghishabestari/KI_MEG/dataframes/tinmeg1_sensors.csv')

In [ ]:
df = pd.read_csv('/Users/payamsadeghishabestari/KI_MEG/dataframes/tinmeg1_sensors.csv')

# Supplementary figure A
order_1 = ['PO60_70', 'PO60_75', 'PO60_80', 'PO60_85', 'PO60_90', 'PO60_95']
order_2 = ['PO60_70', 'PO70_75', 'PO70_80', 'PO70_85', 'PO70_90', 'PO70_95']
mask = df['Stimulus'].isin(order_2)
df1 = df[mask]
fig, ax = plt.subplots(1, 1, figsize=(10,4))
palette_color = ['#1f77b4', '#d62728'] 
sns.boxplot(data=df1, x='Stimulus', y='peak value (30ms)', hue='Ch_name', width=0.8, fill=False, gap=.1, linewidth=2,
            saturation=0.75, palette=palette_color, order=order_2, ax=ax)
sns.stripplot(data=df1, x='Stimulus', y='peak value (30ms)', hue='Ch_name',
            dodge=True, size=3, palette=palette_color, order=order_2, ax=ax, legend=False)
ax.spines['top'].set_visible(False); ax.spines['right'].set_visible(False)
ax.legend(frameon=False, loc='upper left')
ax.set_yticks(np.array([0, 0.5, 1, 1.5])*1e-11)

# supplementary figure B
fig, axs = plt.subplots(1, 1, figsize=(7, 3))
time_array = np.linspace(-300, 300, 151)
stims = ['PO60_70', 'PO60_75', 'PO60_80', 'PO60_85', 'PO60_90', 'PO60_95']
# stims = ['PO70_75', 'PO70_80', 'PO70_85', 'PO70_90', 'PO70_95']
colors = ['#1f77b4', '#d62728']
color = colors[0]
lw = 0.5
for stim in stims[:-1]:
    axs.plot(time_array, abs(grand_ev_dict[stim].get_data(picks=ch_max_left)[0] * 1e15),
            linewidth=lw, label=stim, color=color)
    lw += 0.4
for stim in stims[-1:]:
    axs.plot(time_array, abs(grand_ev_dict[stim].get_data(picks=ch_max_left)[0] * 1e15),
            linewidth=lw, label=stim, color='k')
    
axs.axvspan(50, 150, alpha=0.4, color='lightgrey')
# axs.legend(fontsize=9, frameon=False, bbox_to_anchor=(0.5, 0.1, 0.6, 0.6))
axs.spines['top'].set_visible(False); axs.spines['right'].set_visible(False)
for i in [-200, -100, 100, 200, 300]:
    axs.vlines(i, -1000, 10000, colors='black',linestyles=':', linewidth=0.5)
axs.vlines(0, -1000, 10000, colors='black',linestyles='--')
axs.set_ylabel(f'fT/cm')
axs.set_xlabel(f'Time (ms)')
axs.set_ylim([-1000, 10000])

# supplementary figure B
info = grand_ev_dict['PO60_70'].pick('grad').info
kwargs = dict(eeg=False, coord_frame="mag")
n_chs = len(info['ch_names'])
sensor_colors = np.zeros(shape=(n_chs, 4)) + matplotlib.colors.to_rgba_array('grey', alpha=0.3)
left_idx = info['ch_names'].index(ch_max_left)
right_idx = info['ch_names'].index(ch_max_right)
sensor_colors[left_idx] = matplotlib.colors.to_rgba_array('#1f77b4', alpha=None)
sensor_colors[right_idx] = matplotlib.colors.to_rgba_array('#d62728', alpha=None)

fig = mne.viz.create_3d_figure((600, 600), bgcolor=(255, 255, 255))
mne.viz.plot_alignment(info=info, surfaces='auto', coord_frame='auto',
                    meg='sensors', eeg=False, ecog=True, fig=fig,
                    dbs=False, interaction='terrain', sensor_colors=sensor_colors)

two-way ANOVA test

In [ ]:
## source
df = pd.read_csv("/Users/payamsadeghishabestari/KI_MEG/dataframes/tinmeg1_transverstemporal.csv")
mask = df['subject ID'].isin([697, 750, 853, 841]) # 841 is just for fun
df = df[~mask]
selected_columns = ['subject ID', 'Stimulus', 'Hemisphere', 'area inhibition (30ms)', 'EOG area inhibition']
df = df[selected_columns]

conditions = [
    df['Stimulus'].str.endswith('_i0'),
    df['Stimulus'].str.endswith('_i60'),
    df['Stimulus'].str.endswith('_i120'),
    df['Stimulus'].str.endswith('_i240')
]
choices = ['0', '60', '120', '240']
df['Inter Stimulus Interval'] = np.select(conditions, choices, default=np.nan)
mask = df.apply(lambda row: any(['nan' in str(cell) for cell in row]), axis=1)
df = df[~mask]
df['Pulse level'] = df['Stimulus'].apply(lambda x: '60' if x.startswith('GP60') else ('70' if x.startswith('GP70') else None))
df = df.rename(columns={'subject ID': 'ID', 'area inhibition (30ms)': 'II',
                        'EOG area inhibition': 'EOG_II',
                        'Inter Stimulus Interval': 'ISI',
                        'Pulse level': 'PL'})
df_rh = df[df['Hemisphere'] == 'rh']
df_lh = df[df['Hemisphere'] == 'lh']

## checking for missing values
assert not sum(df.isnull().sum()), "There are missing values" 

## removing outliers
# thr = 3
# z_scores_rh = stats.zscore(df_rh['II'])
# z_scores_lh = stats.zscore(df_lh['II'])
# z_scores_eog = stats.zscore(df_rh['EOG_II'])

# df_rh = df_rh[~(abs(z_scores_rh) > thr)]
# df_lh = df_lh[~(abs(z_scores_lh) > thr)]
# df_eog = df_rh[~(abs(z_scores_eog) > thr)]

## performing two-way ANOVA
model_rh = ols(formula='II ~ ISI + PL + ISI:PL',
            data=df_rh,
            drop_cols=['ID', 'Stimulus', 'Hemisphere', 'EOG_II']).fit()
model_lh = ols(formula='II ~ ISI + PL + ISI:PL',
            data=df_lh,
            drop_cols=['ID', 'Stimulus', 'Hemisphere', 'EOG_II']).fit()
model_eog = ols(formula='EOG_II ~ ISI + PL + ISI:PL',
            data=df_rh,
            drop_cols=['ID', 'Stimulus', 'Hemisphere', 'II']).fit()

## performing LLM
result_llm_rh = smf.mixedlm("II ~ ISI + PL", df_rh, groups=df_rh.index).fit()
result_llm_lh = smf.mixedlm("II ~ ISI + PL", df_lh, groups=df_lh.index).fit()
# result_llm_eog = smf.mixedlm("II ~ ISI + PL", df_eog, groups=df_eog.index).fit()

## checking normality with Shapiro-Wilk test
for model, title in zip([model_rh, model_lh, model_eog], ["rh", "lh", "eog"]):
    residuals = model.resid
    shapiro_test_residuals = stats.shapiro(residuals)
    # print(f"Shapiro-Wilk test for residuals in {title}:", shapiro_test_residuals)

table_rh = sm.stats.anova_lm(model_rh, typ=2)
table_lh = sm.stats.anova_lm(model_lh, typ=2)
table_eog = sm.stats.anova_lm(model_eog, typ=2)

## ANOVA repeated measure
aovrm2way_rh = AnovaRM(df_rh, "II", "ID", within=["ISI", "PL"]).fit()
aovrm2way_lh = AnovaRM(df_lh, "II", "ID", within=["ISI", "PL"]).fit()
aovrm2way_eog = AnovaRM(df_rh, "EOG_II", "ID", within=["ISI", "PL"]).fit()

## tukey multi comparison
tukey_rh = MultiComparison(data=df_rh['II'], groups=df_rh['ISI']).tukeyhsd() # could be changed to PL
tukey_lh = MultiComparison(data=df_lh['II'], groups=df_lh['ISI']).tukeyhsd()
tukey_eog = MultiComparison(data=df_rh['EOG_II'], groups=df_rh['ISI']).tukeyhsd()

In [ ]:
print(table_rh)

In [ ]:
## sensor
df = pd.read_csv("/Users/payamsadeghishabestari/KI_MEG/dataframes/tinmeg1_sensors.csv")
mask = df['subject ID'].isin([697, 750, 853, 841])
df = df[~mask]
selected_columns = ['subject ID', 'Stimulus', 'Ch_name', 'area inhibition (30ms)']
df = df[selected_columns]

conditions = [
    df['Stimulus'].str.endswith('_i0'),
    df['Stimulus'].str.endswith('_i60'),
    df['Stimulus'].str.endswith('_i120'),
    df['Stimulus'].str.endswith('_i240')
]
choices = ['0', '60', '120', '240']
df['Inter Stimulus Interval'] = np.select(conditions, choices, default=np.nan)
mask = df.apply(lambda row: any(['nan' in str(cell) for cell in row]), axis=1)
df = df[~mask]
df['Pulse level'] = df['Stimulus'].apply(lambda x: '60' if x.startswith('GP60') else ('70' if x.startswith('GP70') else None))
df = df.rename(columns={'subject ID': 'ID', 'area inhibition (30ms)': 'II',
                        'Inter Stimulus Interval': 'ISI',
                        'Pulse level': 'PL'})
df_rh = df[df['Ch_name'] == 'MEG1332']
df_lh = df[df['Ch_name'] == 'MEG0242']

## removing outliers
# thr = 3
# z_scores_rh = stats.zscore(df_rh['II'])
# z_scores_lh = stats.zscore(df_lh['II'])

# df_rh = df_rh[~(abs(z_scores_rh) > thr)]
# df_lh = df_lh[~(abs(z_scores_lh) > thr)]

## performing two-way ANOVA and tukey test
model_rh = ols(formula='II ~ C(ISI) + C(PL) + C(ISI):C(PL)',
            data=df_rh,
            drop_cols=['ID', 'Stimulus', 'Ch_name', 'EOG_II']).fit()
model_lh = ols(formula='II ~ C(ISI) + C(PL) + C(ISI):C(PL)',
            data=df_lh,
            drop_cols=['ID', 'Stimulus', 'Ch_name', 'EOG_II']).fit()

table_rh = sm.stats.anova_lm(model_rh, typ=2)
table_lh = sm.stats.anova_lm(model_lh, typ=2)

## ANOVA repeated measure
aovrm2way_rh = AnovaRM(df_rh, "II", "ID", within=["ISI", "PL"]).fit()
aovrm2way_lh = AnovaRM(df_lh, "II", "ID", within=["ISI", "PL"]).fit()

tukey_rh = MultiComparison(data=df_rh['II'], groups=df_rh['ISI']).tukeyhsd() # could be changed to PL
tukey_lh = MultiComparison(data=df_lh['II'], groups=df_lh['ISI']).tukeyhsd()

Scatter Plots

In [ ]:
## source
df = pd.read_csv("/Users/payamsadeghishabestari/KI_MEG/dataframes/tinmeg1_transverstemporal.csv")
mask = df['subject ID'].isin([697, 750, 853, 841]) # 841 is just for fun
df = df[~mask]
selected_columns = ['subject ID', 'Stimulus', 'Hemisphere', 'area inhibition (30ms)', 'EOG area inhibition']
df = df[selected_columns]

conditions = [
    df['Stimulus'].str.endswith('_i0'),
    df['Stimulus'].str.endswith('_i60'),
    df['Stimulus'].str.endswith('_i120'),
    df['Stimulus'].str.endswith('_i240')
]
choices = ['0', '60', '120', '240']
df['Inter Stimulus Interval'] = np.select(conditions, choices, default=np.nan)
mask = df.apply(lambda row: any(['nan' in str(cell) for cell in row]), axis=1)
df = df[~mask]
df['Pulse level'] = df['Stimulus'].apply(lambda x: '60' if x.startswith('GP60') else ('70' if x.startswith('GP70') else None))
df = df.rename(columns={'subject ID': 'ID', 'area inhibition (30ms)': 'II',
                        'EOG area inhibition': 'EOG_II',
                        'Inter Stimulus Interval': 'ISI',
                        'Pulse level': 'PL'})
df_rh = df[df['Hemisphere'] == 'rh']
df_lh = df[df['Hemisphere'] == 'lh']

## checking for missing values
assert not sum(df.isnull().sum()), "There are missing values" 

## ta king only positive values
df_rh = df_rh[df_rh["II"] > 0]
df_rh = df_rh[df_rh["EOG_II"] > 0]

In [ ]:
fig, axs = plt.subplots(2, 4, figsize=(11, 5))
fig.subplots_adjust(hspace=0.5)
axes = list(product(range(2), range(4)))
stims = ["GP60_i0", "GP60_i60", "GP60_i120", "GP60_i240",
        "GP70_i0", "GP70_i60", "GP70_i120", "GP70_i240"]
(x, y) = ("II", "EOG_II")
cmap = plt.get_cmap("Set1")

# plotting the scatter plot
for (i , j), stim in zip(axes, stims):
    
    df_s = df_rh[df["Stimulus"]==stim]
    scatter_kws = {"s": 10, "color": cmap.colors[j]}
    line_kws = {"linestyle": "--", "linewidth": 1, "color": "k"}
    sns.regplot(data=df_s, x=x, y=y, ax=axs[i][j], ci=95, scatter_kws=scatter_kws,
                line_kws=line_kws)
    axs[i][j].set_xlabel("")
    axs[i][j].set_ylabel("")
    axs[i][j].spines[['right', 'top']].set_visible(False)
    axs[i][j].set_xlim([-10, 110])
    axs[i][j].set_ylim([-10, 110])

    # to print p and r values
    X = sm.add_constant(df_s[x]) 
    model = sm.GLS(df_s[y], X)
    results = model.fit()
    axs[i][j].set_title(f"R_const:{round(results.params.const, 3)}, R_ii:{round(results.params.II, 3)}\n p_const:{round(results.pvalues.const, 3)}, p_ii:{round(results.pvalues.II, 3)}", fontsize=6)


axs[0][0].set_ylabel("EOG Inhibition")
axs[1][0].set_ylabel("EOG Inhibition")
# for i, title, xlabel in zip(range(4), ["0 ms", "60 ms", "120 ms", "240 ms"], ["Inhibition Index"] * 4):
#     axs[0][i].set_title(title)
#     axs[1][i].set_xlabel(xlabel)

axs[0][3].yaxis.set_label_position("right")
axs[1][3].yaxis.set_label_position("right")

axs[0][3].set_ylabel("Noise Level 60", rotation=270)
axs[1][3].set_ylabel("Noise Level 70", rotation=270)



Canonical Correlation Check

In [ ]:
## source
df = pd.read_csv("/Users/payamsadeghishabestari/KI_MEG/dataframes/tinmeg1_transverstemporal.csv")
mask = df['subject ID'].isin([697, 750, 853, 841]) # 841 is just for fun
df = df[~mask]
selected_columns = ['subject ID', 'Stimulus', 'Hemisphere', 'area inhibition (30ms)', 'EOG area inhibition']
df = df[selected_columns]

conditions = [
    df['Stimulus'].str.endswith('_i0'),
    df['Stimulus'].str.endswith('_i60'),
    df['Stimulus'].str.endswith('_i120'),
    df['Stimulus'].str.endswith('_i240')
]
choices = ['0', '60', '120', '240']
df['Inter Stimulus Interval'] = np.select(conditions, choices, default=np.nan)
mask = df.apply(lambda row: any(['nan' in str(cell) for cell in row]), axis=1)
df = df[~mask]
df['Pulse level'] = df['Stimulus'].apply(lambda x: '60' if x.startswith('GP60') else ('70' if x.startswith('GP70') else None))
df = df.rename(columns={'subject ID': 'ID', 'area inhibition (30ms)': 'II',
                        'EOG area inhibition': 'EOG_II',
                        'Inter Stimulus Interval': 'ISI',
                        'Pulse level': 'PL'})
df_rh = df[df['Hemisphere'] == 'rh']
df_lh = df[df['Hemisphere'] == 'lh']

## checking for missing values
assert not sum(df.isnull().sum()), "There are missing values" 

In [ ]:
fig, axs = plt.subplots(2, 4, figsize=(11, 5))
fig.subplots_adjust(hspace=0.5)
axes = list(product(range(2), range(4)))
stims = ["GP60_i0", "GP60_i60", "GP60_i120", "GP60_i240",
        "GP70_i0", "GP70_i60", "GP70_i120", "GP70_i240"]
cmap = plt.get_cmap("Set1")
df_rh = df[df['Hemisphere'] == 'rh']
df_lh = df[df['Hemisphere'] == 'lh']

for (i , j), stim in zip(axes, stims):
    # create the dataframe
    df1 = df_rh[df_rh["Stimulus"]==stim]
    df2 = df_lh[df_lh["Stimulus"]==stim]
    new_df_dict = {"Stimulus": [stim]*len(df1),
                    "right_II": np.array(df1["II"]),
                    "left_II": np.array(df2["II"]),
                    "EOG_II": np.array(df1["EOG_II"])}
    df_both = pd.DataFrame(new_df_dict)
    df_both = df_both[(df_both["right_II"]>0) & (df_both["left_II"]>0) & (df_both["EOG_II"]>0)]

    ## CC analysis
    X = df_both[["left_II", "right_II"]]
    Y = df_both["EOG_II"]
    cca = CCA(n_components=1)
    cca.fit(X, Y)
    X_c, Y_c = cca.transform(X, Y)

    ## plotting
    scatter_kws = {"s": 10, "color": cmap.colors[j]}
    line_kws = {"linestyle": "--", "linewidth": 1, "color": "k"}
    sns.regplot(data=df_both, x=X_c, y=Y_c, ax=axs[i][j], ci=95, scatter_kws=scatter_kws,
                line_kws=line_kws)
    axs[i][j].set_xlabel("")
    axs[i][j].set_ylabel("")
    axs[i][j].spines[['right', 'top']].set_visible(False)
    axs[i][j].set_xlim([-2.5, 2.5])
    axs[i][j].set_ylim([-2.5, 2.5])

    # to print p and r values
    sm_x = sm.add_constant(X_c) 
    model = sm.GLS(Y_c, sm_x)
    results = model.fit()
    axs[i][j].set_title(f"R_const:{round(results.params[0], 3)}, R_ii:{round(results.params[1], 3)}\n p_const:{round(results.pvalues[0], 3)}, p_ii:{round(results.pvalues[1], 3)}", fontsize=6)


axs[0][0].set_ylabel("EOG Inhibition")
axs[1][0].set_ylabel("EOG Inhibition")
# for i, title, xlabel in zip(range(4), ["0 ms", "60 ms", "120 ms", "240 ms"], ["Inhibition Index"] * 4):
#     axs[0][i].set_title(title)
#     axs[1][i].set_xlabel(xlabel)

axs[0][3].yaxis.set_label_position("right")
axs[1][3].yaxis.set_label_position("right")

axs[0][3].set_ylabel("Noise Level 60", rotation=270)
axs[1][3].set_ylabel("Noise Level 70", rotation=270)

Pulse Only Correlation

In [ ]:
## source
df = pd.read_csv("/Users/payamsadeghishabestari/KI_MEG/dataframes/tinmeg1_transverstemporal.csv")
mask = df['subject ID'].isin([697, 750, 853, 841]) # 841 is just for fun
df = df[~mask]

In [ ]:
## source
df = pd.read_csv("/Users/payamsadeghishabestari/KI_MEG/dataframes/tinmeg1_transverstemporal.csv")
mask = df['subject ID'].isin([697, 750, 853, 841]) # 841 is just for fun
df = df[~mask]
selected_columns = ['subject ID', 'Stimulus', 'Hemisphere', 'area value (30ms)', 'EOG area (30ms)']
df = df[selected_columns]
df_rh = df[df['Hemisphere'] == 'rh']
df_lh = df[df['Hemisphere'] == 'lh']

In [ ]:
fig, axs = plt.subplots(1, 2, figsize=(6, 2))
fig.subplots_adjust(hspace=0.5)
axes = range(2)
stims = ["PO60_90", "PO70_90"]
(x, y) = ("area value (30ms)", "EOG area (30ms)")
cmap = plt.get_cmap("Set1")

# plotting the scatter plot
for i, stim in zip(axes, stims):
    
    df_s = df_rh[df_rh["Stimulus"]==stim]

    scatter_kws = {"s": 10, "color": cmap.colors[j]}
    line_kws = {"linestyle": "--", "linewidth": 1, "color": "k"}
    sns.regplot(data=df_s, x=x, y=y, ax=axs[i], ci=95, scatter_kws=scatter_kws,
                line_kws=line_kws)
    axs[i].set_xlabel("")
    axs[i].set_ylabel("")
    axs[i].spines[['right', 'top']].set_visible(False)
    axs[i].set_xlim([15, 90])
    axs[i].set_ylim([-10, 750])

    # to print p and r values
    X = sm.add_constant(df_s[x]) 
    model = sm.GLS(df_s[y], X)
    results = model.fit()
    print(results.pvalues)
    # axs[i][j].set_title(f"R_const:{round(results.params.const, 3)}, R_ii:{round(results.params.II, 3)}\n p_const:{round(results.pvalues.const, 3)}, p_ii:{round(results.pvalues.II, 3)}", fontsize=6)


Connectivity Analysis

In [ ]:
# Create epochs dictionary (some needs concatenating)
epochs_folder = '/Users/payamsadeghishabestari/KI_MEG/epochs/tinmeg1'
epochs_file = {}
for f in sorted(os.listdir(epochs_folder)):
    file = os.path.join(epochs_folder, f)
    if file.endswith("-epo.fif") and '697' not in file and '750' not in file and '853' not in file and '841' not in file:
        epochs_file[f'{file[-11:-8]}'] = file

# create dictionary for stc objects
key_stims = ['GP60_i0', 'GP60_i60', 'GP60_i120', 'GP60_i240',
        'GP70_i0', 'GP70_i60', 'GP70_i120', 'GP70_i240'] 
po_stims = ['PO60_70', 'PO60_75', 'PO60_80', 'PO60_85', 'PO60_90', 'PO60_95']
subjects_dir = '/Applications/freesurfer/7.4.1/subjects'
method = "dSPM"
snr = 3.0
lambda2 = 1.0 / snr**2
stcs_dict = {}

for subject in tqdm(list(epochs_file.keys())):
    
    # reading the epoch
    epochs = mne.read_epochs(fname=epochs_file[subject], preload=True, verbose=False)
    
    # setting up the surface source space
    src = mne.setup_source_space(f'0{subject}', spacing="oct6", subjects_dir=subjects_dir, n_jobs=-1, verbose=None)

    # setting up the boundary-element model (BEM) 
    bem_model = mne.make_bem_model(subject=f'0{subject}', ico=4, subjects_dir=subjects_dir, verbose=None)  
    bem = mne.make_bem_solution(bem_model, verbose=False)

    # aligning coordinate frame (coregistration MEG-MRI)
    info = epochs.info 
    coreg = Coregistration(info, f'0{subject}', subjects_dir, fiducials='auto')
    coreg.fit_fiducials(verbose=False)
    coreg.fit_icp(n_iterations=40, nasion_weight=2.0, verbose=False) 
    coreg.omit_head_shape_points(distance=5.0 / 1000) 
    coreg.fit_icp(n_iterations=40, nasion_weight=10, verbose=False) 

    # Computing the forward solution
    print(f'Computing the forward solution ...')
    fwd = mne.make_forward_solution(info, trans=coreg.trans, src=src, bem=bem, meg=True,
                                    eeg=False, mindist=5.0, n_jobs=None, verbose=False)

    # Computing the regularized noise-covariance matrix (consider the notes)
    print(f'Estimate the noise covariance of the recording ...')
    noise_cov = mne.compute_covariance(epochs[po_stims], tmax=0.0, method=("empirical", "shrunk"),
                                        verbose=False)
    
    # Computing the minimum-norm inverse solution
    print(f'Computing the minimum-norm inverse solution ...')
    inverse_operator = make_inverse_operator(info, fwd, noise_cov, loose=0.2, depth=0.8, verbose=False)

    # Compute source estimate object
    print(f'Computing and saving the source estimate object ...')
    stc_dict = {}
    for key in key_stims:
        stc = apply_inverse_epochs(epochs[key], inverse_operator, lambda2,
                                    method=method, pick_ori=None, verbose=False)
        stc_dict[key] = stc
        
    stcs_dict[subject] = stc_dict


fname_fsaverage_src = '/Users/payamsadeghishabestari/mne_data/MNE-fsaverage-data/fsaverage/bem/fsaverage-ico-5-src.fif'
src_to = mne.read_source_spaces(fname_fsaverage_src)
labels = mne.read_labels_from_annot(subject='fsaverage', parc="aparc", subjects_dir=None, verbose=False)[:-1]

## create dictionary for labels_ts
labels_tss_dict = {} 
for subject in tqdm(list(stcs_dict.keys())):
    labels_ts_dict = {}
    for key in key_stims:
        ## morph them to fsaverage
        stc = stcs_dict[subject][key]
        morph = mne.compute_source_morph(stc[0], subject_from=f'0{subject}', subject_to="fsaverage",
                                        subjects_dir=subjects_dir, src_to=src_to)
        stc_morph = []
        for stc_ep in stc: 
            stc_morph.append(morph.apply(stc_ep))
        
        ## extract time labels
        labels_ts = mne.extract_label_time_course(stc_morph, labels, src=src_to,
                                                return_generator=False, verbose=False)
        labels_ts_dict[key] = np.array(labels_ts)
    labels_tss_dict[subject] = labels_ts_dict

## save the dictionary
file_path = '/Users/payamsadeghishabestari/KI_MEG/stcs/labels_ts_dict.pickle'
with open(file_path, 'wb') as file:
    pickle.dump(labels_tss_dict, file)

In [ ]:
# load the dictionary and compute connectivity
file_path = '/Users/payamsadeghishabestari/KI_MEG/stcs/labels_ts_dict.pickle'
with open(file_path, 'rb') as file:
    labels_ts_dict = pickle.load(file)

key_stims = ['GP60_i0', 'GP60_i60', 'GP60_i120', 'GP60_i240',
        'GP70_i0', 'GP70_i60', 'GP70_i120', 'GP70_i240'] 
labels = mne.read_labels_from_annot(subject='fsaverage', parc="aparc", subjects_dir=None, verbose=False)[:-1]
lb_names = [lb.name for lb in labels]
sfreq = 250
(tmin, tmax) = (0.3, None)
method = "wpli" # I would suggest pli, wpli or plv
mode = "multitaper"
Freq_Bands = {"theta": [4.0, 8.0], "alpha": [8.0, 13.0], "beta": [13.0, 30.0]}
min_freq = np.min(list(Freq_Bands.values()))
max_freq = np.max(list(Freq_Bands.values()))
freqs = np.linspace(min_freq, max_freq, int((max_freq - min_freq) * 4 + 1))
fmins = tuple([list(Freq_Bands.values())[f][0] for f in range(len(Freq_Bands))])
fmaxs = tuple([list(Freq_Bands.values())[f][1] for f in range(len(Freq_Bands))])
low_tri_idxs = np.tril_indices(len(labels), k=1)

con_dict = {}
for subject in tqdm(list(labels_ts_dict.keys())):
    con_sub_dict = {}
    for key in key_stims:
        con_sub_sub_dict = {}
        data = labels_ts_dict[subject][key]
        for fmin, fmax, freq_band in zip(fmins, fmaxs, list(Freq_Bands.keys())):
            con = spectral_connectivity_epochs(data=data, names=lb_names, method=method, sfreq=sfreq,
                                                mode=mode, fmin=fmin, fmax=fmax, cwt_freqs=freqs, 
                                                faverage=False, tmin=tmin, tmax=tmax, verbose=False)
            con_data = con.get_data(output="dense")[:,:,0]
            con_sub_sub_dict[freq_band] = con_data[low_tri_idxs]
        con_sub_dict[key] = con_sub_sub_dict
    con_dict[subject] = con_sub_dict

# reshape the dictionary first
nl_dict = {f: {} for f in list(Freq_Bands.keys())}
for key in list(nl_dict.keys()):
    nl_dict[key] = {'GP60_i0': [], 'GP60_i60': [], 'GP60_i120': [], 'GP60_i240': [],
        'GP70_i0': [], 'GP70_i60': [], 'GP70_i120': [], 'GP70_i240': []}

for freq_band in list(Freq_Bands.keys()):
    for key in key_stims:
        for subject in list(con_dict.keys()):
            nl_dict[freq_band][key].append(con_dict[subject][key][freq_band])

In [ ]:
## stats (change indexes to compare)
freq_band = "alpha"
mode = "average" # or average
(p_thr, alpha) = (0.05, 0.05)
keys1 = np.array(key_stims)[[1, 5]]
keys2 = np.array(key_stims)[[3, 7]]


nl_60_data = []
nl_70_data = []
for key in keys1:
    nl_60_data.append(np.array(nl_dict[freq_band][key]))
for key in keys2:
    nl_70_data.append(np.array(nl_dict[freq_band][key]))

if mode == "average":
    nl_60_data = np.array(nl_60_data).mean(axis=0)
    nl_70_data = np.array(nl_70_data).mean(axis=0)
if mode == "concat":
    nl_60_data = np.concatenate(nl_60_data, axis=0)
    nl_70_data = np.concatenate(nl_70_data, axis=0)

stat, p_values = ttest_ind(nl_60_data, nl_70_data, permutations=0, random_state=42)
p_values[np.where(p_values.astype(str)=="nan")[0]] = 0
reject_null, p_corrected, _, _ = sm.stats.multipletests(pvals=p_values, alpha=alpha,
                                                            method="fdr_bh")
label_idxs = []
indexes = [i for i, val in enumerate(p_corrected) if val < p_thr and val != 0]
if len(indexes) == 0:
    print(f"No statistical difference between brain labels for method {method}")
else:
    for idx in indexes:
        label_idxs.append((low_tri_idxs[0][idx], low_tri_idxs[1][idx]))

In [ ]:
# visualization
n_labels = len(labels)
control_avg_vector = np.mean(nl_60_data, axis=0)
control_avg_graph = np.zeros(shape=(n_labels, n_labels))
control_avg_graph[low_tri_idxs] = control_avg_vector
control_avg_graph = control_avg_graph + control_avg_graph.T

case_avg_vector = np.mean(nl_70_data, axis=0)
case_avg_graph = np.zeros(shape=(n_labels, n_labels))
case_avg_graph[low_tri_idxs] = case_avg_vector
case_avg_graph = case_avg_graph + case_avg_graph.T

diff_graph = control_avg_graph - case_avg_graph

node_coords = []
for label in labels:
    if label.hemi == 'lh':
        hemi = 0
    if label.hemi == 'rh':
        hemi = 1
    center_vertex = label.center_of_mass(subject='fsaverage', 
                                        restrict_vertices=False, 
                                        subjects_dir=None)
    mni_pos = mne.vertex_to_mni(center_vertex, hemis=hemi,
                            subject='fsaverage', subjects_dir=None)
    node_coords.append(mni_pos)

node_coords = np.array(node_coords)
ticks = [lb.name for lb in labels]

diff_graph = control_avg_graph - case_avg_graph
zero_matrix = np.zeros(shape=(n_labels, n_labels))
for i, j in label_idxs:
    zero_matrix[i][j] = diff_graph[i][j]
stat_graph = zero_matrix + zero_matrix.T  

fig, ax = plt.subplots(1, 1, figsize=(11, 5))
plot_connectome(adjacency_matrix=stat_graph, node_coords=node_coords, display_mode="lzry",
                node_color='k', node_size=20, axes=ax, colorbar=False,
                edge_threshold="99%")
fig.tight_layout()

Some ANOVA

In [ ]:
## preparing the dataframe for anova pulse only
df = pd.read_csv("/Users/payamsadeghishabestari/KI_MEG/dataframes/tinmeg1_transverstemporal.csv")
# remove some subjects
mask = df['subject ID'].isin([697, 750, 853, 841]) # 841 is just for fun
df = df[~mask]
# take only pulse only
mask_po = df["Stimulus"].isin(["PO60_75", "PO60_80", "PO60_85", "PO60_90", "PO60_95",
                                "PO70_75", "PO70_80", "PO70_85", "PO70_90", "PO70_95"])
df = df[mask_po]
# create two new columns PL and BBN
def check_BBN(string):
    if 'PO60' in string:
        return 60
    elif 'PO70' in string:
        return 70
def check_PL(string):
    return string[-2:]   
df['BBN'] = df['Stimulus'].apply(check_BBN)
df['PL'] = df['Stimulus'].apply(check_PL)
# rename latency columns
df = df.rename(columns={"EOG peak latency": "EOG_peak_latency", "EOG peak": "EOG_peak",
                        "peak value (30ms)": "peak_value_30ms", "subject ID": "ID"})
df_eog = df[df["Hemisphere"]=="rh"]
df_rh = df[df["Hemisphere"]=="rh"]
df_lh = df[df["Hemisphere"]=="lh"]

In [ ]:
# eog latency anova
model_eog_lat = ols(formula='EOG_peak_latency ~ C(PL) + C(BBN) + C(PL):C(BBN)',
            data=df_eog).fit()
table_eog_lat = sm.stats.anova_lm(model_eog_lat, typ=2)
aovrm2way_eog_lat = AnovaRM(df_eog, "EOG_peak_latency", "ID", within=["PL", "BBN"]).fit()

# lh latency anova
model_lh_lat = ols(formula='latency ~ C(PL) + C(BBN) + C(PL):C(BBN)',
            data=df_lh).fit()
table_lh_lat = sm.stats.anova_lm(model_lh_lat, typ=2)
aovrm2way_lh_lat = AnovaRM(df_lh, "latency", "ID", within=["PL", "BBN"]).fit()

# rh latency anova
model_rh_lat = ols(formula='latency ~ C(PL) + C(BBN) + C(PL):C(BBN)',
            data=df_rh).fit()
table_rh_lat = sm.stats.anova_lm(model_rh_lat, typ=2)
aovrm2way_rh_lat = AnovaRM(df_rh, "latency", "ID", within=["PL", "BBN"]).fit()

# eog peak amplitude
model_eog_peak = ols(formula='EOG_peak ~ C(PL) + C(BBN) + C(PL):C(BBN)',
            data=df_eog).fit()
table_eog_peak = sm.stats.anova_lm(model_eog_peak, typ=2)
aovrm2way_eog_peak = AnovaRM(df_eog, "EOG_peak", "ID", within=["PL", "BBN"]).fit()

# lh peak amplitude
model_lh_peak = ols(formula='peak_value_30ms ~ C(PL) + C(BBN) + C(PL):C(BBN)',
            data=df_lh).fit()
table_lh_peak = sm.stats.anova_lm(model_lh_peak, typ=2)
aovrm2way_lh_lat = AnovaRM(df_lh, "peak_value_30ms", "ID", within=["PL", "BBN"]).fit()

# rh peak amplitude
model_rh_peak = ols(formula='peak_value_30ms ~ C(PL) + C(BBN) + C(PL):C(BBN)',
            data=df_rh).fit()
table_rh_peak = sm.stats.anova_lm(model_rh_peak, typ=2)
aovrm2way_rh_lat = AnovaRM(df_rh, "peak_value_30ms", "ID", within=["PL", "BBN"]).fit()

In [ ]:
print(table_eog_lat)

In [ ]:
df_sub = df_rh[df_rh["BBN"]==60]
tukey = MultiComparison(data=df_sub['peak_value_30ms'], groups=df_sub['PL']).tukeyhsd()
tukey.summary()

ANOVA on Gap Only

In [ ]:
## preparing the dataframe for anova gap only
df = pd.read_csv("/Users/payamsadeghishabestari/KI_MEG/dataframes/tinmeg1_transverstemporal.csv")
# remove some subjects
mask = df['subject ID'].isin([697, 750, 853, 841]) # 841 is just for fun
df = df[~mask]
# take only pulse only
mask_po = df["Stimulus"].isin(["GO_60", "GO_70"])
df = df[mask_po]
# create two new columns PL and BBN
def check_BBN(string):
    if '60' in string:
        return 60
    elif '70' in string:
        return 70
df['BBN'] = df['Stimulus'].apply(check_BBN)
# rename columns
df = df.rename(columns={"peak value (30ms)": "peak_value_30ms", "subject ID": "ID"})

df_rh_60 = df[(df["Hemisphere"]=="rh") & (df["BBN"]==60)]
df_lh_60 = df[(df["Hemisphere"]=="lh") & (df["BBN"]==60)]
df_rh_70 = df[(df["Hemisphere"]=="rh") & (df["BBN"]==70)]
df_lh_70 = df[(df["Hemisphere"]=="lh") & (df["BBN"]==70)]

print(f_oneway(df_rh_60["peak_value_30ms"], df_lh_60["peak_value_30ms"]))
print(f_oneway(df_rh_70["peak_value_30ms"], df_lh_70["peak_value_30ms"]))

##
print(f_oneway(df_rh_60["EOG peak"], df_rh_70["EOG peak"]))
print(f_oneway(df_lh_60["peak_value_30ms"], df_lh_70["peak_value_30ms"]))
print(f_oneway(df_rh_60["peak_value_30ms"], df_rh_70["peak_value_30ms"]))

In [ ]:
print(f_oneway(df_rh_60["EOG peak latency"], df_rh_70["EOG peak latency"]))
print(f_oneway(df_lh_60["latency"], df_lh_70["latency"]))
print(f_oneway(df_rh_60["latency"], df_rh_70["latency"]))

In [ ]:
f_oneway(df_rh_60["EOG peak"], df_rh_70["EOG peak"])

In [ ]:
mean = round(df_rh_70["latency"].mean(), 2)
std = round(df_rh_70["latency"].std(), 1)
print(f"{mean}, {std}")

In [ ]:
## preparing the dataframe for scatter plot
df = pd.read_csv("/Users/payamsadeghishabestari/KI_MEG/dataframes/tinmeg1_transverstemporal.csv")
# remove some subjects
mask = df['subject ID'].isin([697, 750, 853, 841]) # 841 is just for fun
df = df[~mask]
df_rh_60 = df[(df["Stimulus"]=="PO60_90") & (df["Hemisphere"]=="rh")]
df_lh_60 = df[(df["Stimulus"]=="PO60_90") & (df["Hemisphere"]=="lh")]
df_rh_70 = df[(df["Stimulus"]=="PO70_90") & (df["Hemisphere"]=="rh")]
df_lh_70 = df[(df["Stimulus"]=="PO70_90") & (df["Hemisphere"]=="lh")]

In [ ]:
fig, axs = plt.subplots(1, 2, figsize=(11, 5))
kwargs={"color": "b"}
sns.scatterplot(data=None, x=df_lh_60["peak value (30ms)"], y=df_lh_60["EOG peak"], ax=axs[0], **kwargs)
kwargs={"color": "r"}
sns.scatterplot(data=None, x=df_rh_60["peak value (30ms)"], y=df_rh_60["EOG peak"], ax=axs[0], **kwargs)

kwargs={"color": "b"}
sns.scatterplot(data=None, x=df_lh_70["peak value (30ms)"], y=df_lh_70["EOG peak"], ax=axs[1], **kwargs)
kwargs={"color": "r"}
sns.scatterplot(data=None, x=df_rh_70["peak value (30ms)"], y=df_rh_70["EOG peak"], ax=axs[1], **kwargs)

axs[0].set_title("BBN 60")
axs[1].set_title("BBN 70")
axs[0].set_ylim([0, 80])
axs[1].set_ylim([0, 80])

Connectivity in Gap only between right and left transverse temporal (and all)

In [ ]:
# Create epochs dictionary (some needs concatenating)
epochs_folder = '/Users/payamsadeghishabestari/KI_MEG/epochs/tinmeg1'
epochs_file = {}
for f in sorted(os.listdir(epochs_folder)):
    file = os.path.join(epochs_folder, f)
    if file.endswith("-epo.fif") and '697' not in file and '750' not in file and '853' not in file and '841' not in file:
        epochs_file[f'{file[-11:-8]}'] = file

# create dictionary for stc objects
key_stims = ["GO_60", "GO_70"] 
po_stims = ['PO60_70', 'PO60_75', 'PO60_80', 'PO60_85', 'PO60_90', 'PO60_95']
subjects_dir = '/Applications/freesurfer/7.4.1/subjects'
method = "dSPM"
snr = 3.0
lambda2 = 1.0 / snr**2
stcs_dict = {}

for subject in tqdm(list(epochs_file.keys())):
    
    # reading the epoch
    epochs = mne.read_epochs(fname=epochs_file[subject], preload=True, verbose=False)
    
    # setting up the surface source space
    src = mne.setup_source_space(f'0{subject}', spacing="oct6", subjects_dir=subjects_dir, n_jobs=-1, verbose=None)

    # setting up the boundary-element model (BEM) 
    bem_model = mne.make_bem_model(subject=f'0{subject}', ico=4, subjects_dir=subjects_dir, verbose=None)  
    bem = mne.make_bem_solution(bem_model, verbose=False)

    # aligning coordinate frame (coregistration MEG-MRI)
    info = epochs.info 
    coreg = Coregistration(info, f'0{subject}', subjects_dir, fiducials='auto')
    coreg.fit_fiducials(verbose=False)
    coreg.fit_icp(n_iterations=40, nasion_weight=2.0, verbose=False) 
    coreg.omit_head_shape_points(distance=5.0 / 1000) 
    coreg.fit_icp(n_iterations=40, nasion_weight=10, verbose=False) 

    # Computing the forward solution
    print(f'Computing the forward solution ...')
    fwd = mne.make_forward_solution(info, trans=coreg.trans, src=src, bem=bem, meg=True,
                                    eeg=False, mindist=5.0, n_jobs=None, verbose=False)

    # Computing the regularized noise-covariance matrix (consider the notes)
    print(f'Estimate the noise covariance of the recording ...')
    noise_cov = mne.compute_covariance(epochs[po_stims], tmax=0.0, method=("empirical", "shrunk"),
                                        verbose=False)
    
    # Computing the minimum-norm inverse solution
    print(f'Computing the minimum-norm inverse solution ...')
    inverse_operator = make_inverse_operator(info, fwd, noise_cov, loose=0.2, depth=0.8, verbose=False)

    # Compute source estimate object
    print(f'Computing and saving the source estimate object ...')
    stc_dict = {}
    for key in key_stims:
        stc = apply_inverse_epochs(epochs[key], inverse_operator, lambda2,
                                    method=method, pick_ori=None, verbose=False)
        stc_dict[key] = stc
        
    stcs_dict[subject] = stc_dict


fname_fsaverage_src = '/Users/payamsadeghishabestari/mne_data/MNE-fsaverage-data/fsaverage/bem/fsaverage-ico-5-src.fif'
src_to = mne.read_source_spaces(fname_fsaverage_src)
labels = mne.read_labels_from_annot(subject='fsaverage', parc="aparc", subjects_dir=None, verbose=False)[:-1]

## create dictionary for labels_ts
labels_tss_dict = {} 
for subject in tqdm(list(stcs_dict.keys())):
    labels_ts_dict = {}
    for key in key_stims:
        ## morph them to fsaverage
        stc = stcs_dict[subject][key]
        morph = mne.compute_source_morph(stc[0], subject_from=f'0{subject}', subject_to="fsaverage",
                                        subjects_dir=subjects_dir, src_to=src_to)
        stc_morph = []
        for stc_ep in stc: 
            stc_morph.append(morph.apply(stc_ep))
        
        ## extract time labels
        labels_ts = mne.extract_label_time_course(stc_morph, labels, src=src_to,
                                                return_generator=False, verbose=False)
        labels_ts_dict[key] = np.array(labels_ts)
    labels_tss_dict[subject] = labels_ts_dict

## save the dictionary
file_path = '/Users/payamsadeghishabestari/KI_MEG/stcs/labels_ts_go_dict.pickle'
with open(file_path, 'wb') as file:
    pickle.dump(labels_tss_dict, file)

In [ ]:
# load the dictionary and compute connectivity
file_path = '/Users/payamsadeghishabestari/KI_MEG/stcs/labels_ts_go_dict.pickle'
with open(file_path, 'rb') as file:
    labels_ts_dict = pickle.load(file)

In [ ]:
data = labels_ts_dict["539"]["GO_60"]

In [ ]:
# load the dictionary and compute connectivity
file_path = '/Users/payamsadeghishabestari/KI_MEG/stcs/labels_ts_go_dict.pickle'
with open(file_path, 'rb') as file:
    labels_ts_dict = pickle.load(file)

key_stims = ["GO_60", "GO_70"] 
labels = mne.read_labels_from_annot(subject='fsaverage', parc="aparc", subjects_dir=None, verbose=False)[:-1]
lb_names = [lb.name for lb in labels]
sfreq = 250
(tmin, tmax) = (0.3, None)
method = "wpli" # I would suggest pli, wpli or plv
mode = "multitaper"
Freq_Bands = {"theta": [4.0, 8.0], "alpha": [8.0, 13.0], "beta": [13.0, 30.0], "gamma": [30.0, 80.0]}
min_freq = np.min(list(Freq_Bands.values()))
max_freq = np.max(list(Freq_Bands.values()))
freqs = np.linspace(min_freq, max_freq, int((max_freq - min_freq) * 4 + 1))
fmins = tuple([list(Freq_Bands.values())[f][0] for f in range(len(Freq_Bands))])
fmaxs = tuple([list(Freq_Bands.values())[f][1] for f in range(len(Freq_Bands))])
low_tri_idxs = np.tril_indices(len(labels), k=-1)

con_dict = {}
for subject in tqdm(list(labels_ts_dict.keys())):
    con_sub_dict = {}
    for key in key_stims:
        con_sub_sub_dict = {}
        data = labels_ts_dict[subject][key]
        for fmin, fmax, freq_band in zip(fmins, fmaxs, list(Freq_Bands.keys())):
            con = spectral_connectivity_epochs(data=data, names=lb_names, method=method, sfreq=sfreq,
                                                mode=mode, fmin=fmin, fmax=fmax, cwt_freqs=freqs, 
                                                faverage=True, tmin=tmin, tmax=tmax, verbose=False)
            con_data = con.get_data(output="dense")[:,:,0]
            con_sub_sub_dict[freq_band] = con_data[low_tri_idxs]
        con_sub_dict[key] = con_sub_sub_dict
    con_dict[subject] = con_sub_dict

# reshape the dictionary first
nl_dict = {f: {} for f in list(Freq_Bands.keys())}
for key in list(nl_dict.keys()):
    nl_dict[key] = {"GO_60": [], "GO_70": []}

for freq_band in list(Freq_Bands.keys()):
    for key in key_stims:
        for subject in list(con_dict.keys()):
            nl_dict[freq_band][key].append(con_dict[subject][key][freq_band])

In [ ]:
## stats (change indexes to compare)
freq_band = "beta"
mode = "average" # or average
(p_thr, alpha) = (0.05, 0.05)
keys1 = np.array(key_stims)[[0]]
keys2 = np.array(key_stims)[[1]]
idx1 = lb_names.index("transversetemporal-lh")
idx2 = lb_names.index("transversetemporal-rh")

nl_60_data = []
nl_70_data = []
for key in keys1:
    nl_60_data.append(np.array(nl_dict[freq_band][key]))
for key in keys2:
    nl_70_data.append(np.array(nl_dict[freq_band][key]))

if mode == "average":
    nl_60_data = np.array(nl_60_data).mean(axis=0)
    nl_70_data = np.array(nl_70_data).mean(axis=0)
if mode == "concat":
    nl_60_data = np.concatenate(nl_60_data, axis=0)
    nl_70_data = np.concatenate(nl_70_data, axis=0)

stat, p_values = ttest_ind(nl_60_data, nl_70_data, permutations=0, random_state=42)
p_values[np.where(p_values.astype(str)=="nan")[0]] = 0

index = np.where((low_tri_idxs[0]== idx2) & (low_tri_idxs[1] == idx1))[0][0]
print(p_values[index])

reject_null, p_corrected, _, _ = sm.stats.multipletests(pvals=p_values, alpha=alpha,
                                                            method="fdr_bh")
label_idxs = []
indexes = [i for i, val in enumerate(p_corrected) if val < p_thr and val != 0]
if len(indexes) == 0:
    print(f"No statistical difference between brain labels for method {method}")
else:
    for idx in indexes:
        label_idxs.append((low_tri_idxs[0][idx], low_tri_idxs[1][idx]))

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(10,4))
palette_color = ['#1f77b4', '#d62728'] 
sns.boxplot(data=[nl_60_data[:,index], nl_70_data[:,index]], width=0.8, fill=False, gap=.1, linewidth=2,
            saturation=0.75, palette=palette_color, ax=ax)
sns.stripplot(data=[nl_60_data[:,index], nl_70_data[:,index]],
            dodge=False, size=3, palette=palette_color, ax=ax, legend=False)
ax.spines['top'].set_visible(False); ax.spines['right'].set_visible(False)

Did Pulse level elicited TT activity?

In [ ]:
#### eog check
fig, axs = plt.subplots(1, 1, figsize=(6, 3))
time_array = np.linspace(-300, 300, 151)
stims = ['PO60_70', 'PO60_75', 'PO60_80', 'PO60_85', 'PO60_90', 'PO60_95']
color = '#ff7f0e'
lw = 0.5
for stim in stims[:-1]:
    axs.plot(time_array, grand_ev_dict[stim].get_data(picks='EOG002')[0] * 1e6,
            linewidth=lw, label=stim, color=color)
    lw += 0.4
for stim in stims[-1:]:
    axs.plot(time_array, grand_ev_dict[stim].get_data(picks='EOG002')[0] * 1e6,
            linewidth=lw, label=stim, color='k')
    
axs.axvspan(50, 180, alpha=0.4, color='lightgrey')
axs.legend(fontsize=9, frameon=False, bbox_to_anchor=(0.5, 0.1, 0.6, 0.6))
axs.spines['top'].set_visible(False); axs.spines['right'].set_visible(False)
for i in [-200, -100, 100, 200, 300]:
    axs.vlines(i, -10, 60, colors='black',linestyles=':', linewidth=0.5)
axs.vlines(0, -10, 60, colors='black',linestyles='--')
axs.set_ylabel(f'EOG amplitude at 70 dB (µv)')
axs.set_xlabel(f'Time (ms)')
axs.set_ylim([-10, 60])

In [ ]:
# eog check
stim = "PO70_80"
scale = 10
po_data = grand_ev_dict[stim].get_data(picks='EOG002')[0] * 1e6
baseline_mean = po_data[25:75].mean()
baseline_mad = np.mean(np.abs(po_data[25:75] - baseline_mean))
po_data_peak = po_data[75:].min()
if po_data_peak < (baseline_mean - scale * baseline_mad):
    print(f"{stim} elucidated eog response.") 
else:
    print(f"{stim} did not elucidate eog response.") 

In [ ]:
# TT check
directory = '/Users/payamsadeghishabestari/KI_MEG/stcs/tinmeg1_morphed'
stc_files_list_rh = {'PO60_70': [], 'PO60_75': [], 'PO60_80': [],
                'PO70_75': [], 'PO70_80': []}
stc_files_list_lh = {'PO60_70': [], 'PO60_75': [], 'PO60_80': [],
                'PO70_75': [], 'PO70_80': []}
events = ['PO60_70', 'PO60_75', 'PO60_80', 'PO70_75', 'PO70_80']

brain_labels = mne.read_labels_from_annot(subject='fsaverage', parc='aparc')[:-1] # 68 labels
fs_dir = fetch_fsaverage(verbose=False)
src_fname = op.join(fs_dir, "bem", "fsaverage-ico-5-src.fif")
src = mne.read_source_spaces(src_fname, verbose=False)

for event in events:
    for filename in sorted(os.listdir(directory)): 
        f = os.path.join(directory, filename)
        if os.path.isfile(f) and f.endswith("-lh.stc") and event in f: # or -rh
                stc = mne.read_source_estimate(fname=f, subject='fsaverage')
                rh_data = stc.extract_label_time_course(brain_labels[-1], src, mode='mean', verbose=False)
                lh_data = stc.extract_label_time_course(brain_labels[-2], src, mode='mean', verbose=False)
                stc_files_list_rh[event].append(rh_data)
                stc_files_list_lh[event].append(lh_data)

In [ ]:
# TT check
stim = "PO70_75"
scale = 10
tt_data = np.squeeze(np.array(stc_files_list_lh[stim])).mean(axis=0)
baseline_mean = tt_data[25:75].mean()
baseline_mad = np.mean(np.abs(tt_data[25:75] - baseline_mean))
tt_data_peak = tt_data[75:].max()
if tt_data_peak > (baseline_mean + scale * baseline_mad):
    print(f"{stim} elucidated eog response.") 
else:
    print(f"{stim} did not elucidate eog response.") 

whether you capture evoked responses even in the non-blinkers?

In [ ]:
for sub_id in list(epochs_file.keys()):
    if sub_id == ["861", "856", "850", "847", "845", "836", "750"]:
        ev = mne.read_epochs(fname=epochs_file[sub_id], verbose=False).average(picks=['eog'], by_event_type=True)
        ev[4].plot(picks="EOG002", titles=sub_id)

In [ ]:
sub_id = "750"
stc_files_list_rh = {'PO60_90': []}
stc_files_list_lh = {'PO60_90': []}
events = ['PO60_90']

brain_labels = mne.read_labels_from_annot(subject='fsaverage', parc='aparc')[:-1] # 68 labels
fs_dir = fetch_fsaverage(verbose=False)
src_fname = op.join(fs_dir, "bem", "fsaverage-ico-5-src.fif")
src = mne.read_source_spaces(src_fname, verbose=False)
directory = '/Users/payamsadeghishabestari/KI_MEG/stcs/tinmeg1_morphed' 

for event in events:
    for filename in sorted(os.listdir(directory)): 
        f = os.path.join(directory, filename)
        if os.path.isfile(f) and f.endswith("-lh.stc") and event in f: # or -rh
            if f[58:61]==sub_id:
                stc = mne.read_source_estimate(fname=f, subject='fsaverage')
                rh_data = stc.extract_label_time_course(brain_labels[-1], src, mode='mean', verbose=False)
                lh_data = stc.extract_label_time_course(brain_labels[-2], src, mode='mean', verbose=False)
                fig, axs = plt.subplots(1, 2, figsize=(12,3))
                colors = ['#1f77b4', '#d62728']
                axs[0].plot(np.linspace(-300, 300, 151), lh_data[0], label=event, color=colors[0])
                axs[1].plot(np.linspace(-300, 300, 151), rh_data[0], label=event, color=colors[1])
                axs[0].set_ylim([-1, 10])
                axs[1].set_ylim([-1, 10])
                axs[0].set_ylabel(f"{sub_id}")